First, create a new conda environment named BI2025 and install the required packages from requirements.txt


In [1]:
#!conda create -n BI2025 python=3.11 -y
#!conda activate BI2025
#!pip install -r requirements.txt

Jupyter detected...
2 channel Terms of Service accepted
Channels:
 - defaults
Platform: osx-arm64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 25.5.1
    latest version: 25.11.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /opt/anaconda3/envs/BI2025

  added / updated specs:
    - python=3.11


The following NEW packages will be INSTALLED:

  bzip2              pkgs/main/osx-arm64::bzip2-1.0.8-h80987f9_6 
  ca-certificates    pkgs/main/osx-arm64::ca-certificates-2025.12.2-hca03da5_0 
  expat              pkgs/main/osx-arm64::expat-2.7.3-h50f4ffc_4 
  libcxx             pkgs/main/osx-arm64::libcxx-20.1.8-hd7fd590_1 
  libexpat           pkgs/main/osx-arm64::libexpat-2.7.3-h50f4ffc_4 
  libffi             pkgs/main/osx-arm64::libffi-3.4.4-hca03da5_1 
  libzlib            pkgs/main/osx-arm64::libzlib-1.3.1-h5f15de7_0 
  ncurses            pkgs/main/osx-arm6

In [1]:
# DO NOT MODIFY OR COPY THIS CELL!! 
# Note: The only imports allowed are Python's standard library, pandas, numpy, scipy, matplotlib, seaborn and scikit-learn
import numpy as np
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
import plotly.express as px
import datetime
import typing
import requests
import time
import shutil
import json
from starvers.starvers import TripleStoreEngine

## Graph-based documentation preliminaries

**!!!IMPORTANT!!!**

Everytime you work on this notebook, enter your student ID in the `executed_by` variable so that the cell executions are accredited to you.

In [2]:
executed_by ='stud-id_12435655'  # Replace the digits after "id_" with your own student ID

Set your group and student IDs. Do this only once.

In [3]:
# group id for this project
group_id = '74'  # Replace the digits with your group id

# Students working on this notebook
student_a = 'stud-id_12435655'  # Replace the digits after "id_" with student A's student ID
student_b = 'stud-id_01556207'  # Replace the digits after "id_" with student B's student ID

In [4]:
# Roles. Don't change these values.
code_writer_role = 'code_writer'
code_executor_role = 'code_executor'

Setup the starvers API for logging your steps into our server-sided graph database.

In [5]:
get_endpoint = "https://starvers.ec.tuwien.ac.at/BI2025"
post_endpoint = "https://starvers.ec.tuwien.ac.at/BI2025/statements"
engine = TripleStoreEngine(get_endpoint, post_endpoint, skip_connection_test=True)

Use these prefixes in your notebooks. You can extend this dict with your prefixes of additional ontologies that you use in this notebook. Replace 00 with your group id

In [6]:
prefixes = {
    'xsd': 'http://www.w3.org/2001/XMLSchema#',
    'rdfs': 'http://www.w3.org/2000/01/rdf-schema#',
    'foaf': 'http://xmlns.com/foaf/0.1/',
    'prov': 'http://www.w3.org/ns/prov#',
    'sc': 'https://schema.org/',
    'cr': 'http://mlcommons.org/croissant/',
    'mls': 'http://www.w3.org/ns/mls#',
    'mlso': 'http://w3id.org/mlso',
    'siu': 'https://si-digital-framework.org/SI/units/',
    'siq': 'https://si-digital-framework.org/SI/quantities/',
    'qudt': 'http://qudt.org/schema/qudt/',
    '': f'https://starvers.ec.tuwien.ac.at/BI2025/{group_id}/',
}

prefix_header = '\n'.join([f'PREFIX {k}: <{v}>' for k, v in prefixes.items()]) + '\n\n'

Ontologies to use
* Provenance of the experiment process
    * PROV-O: 
        * doc: https://www.w3.org/TR/prov-o/
        * serialization: https://www.w3.org/ns/prov-o
* Data used and created
    * schema.org - Dataset: 
        * doc: https://schema.org/Dataset
        * serialization: https://schema.org/version/latest/schemaorg-current-https.ttl
    * Crossaint
        * doc: https://docs.mlcommons.org/croissant/docs/croissant-spec.html
        * serialization: https://github.com/mlcommons/croissant/blob/main/docs/croissant.ttl
* ML experiments performed
    * MLSO: 
        * doc: https://github.com/dtai-kg/MLSO
        * doc: https://dtai-kg.github.io/MLSO/#http://w3id.org/
        * serialization: https://dtai-kg.github.io/MLSO/ontology.ttl
* Measurements, Metrics, Units
    * QUDT
        * doc:https://qudt.org/
        * doc: https://github.com/qudt/qudt-public-repo
        * serialization: https://github.com/qudt/qudt-public-repo/blob/main/src/main/rdf/schema/SCHEMA_QUDT.ttl
    * SI Digital Framework
        * doc: https://github.com/TheBIPM/SI_Digital_Framework/blob/main/SI_Reference_Point/docs/README.md
        * doc: https://si-digital-framework.org/
        * doc: https://si-digital-framework.org/SI
        * serialization: https://github.com/TheBIPM/SI_Digital_Framework/blob/main/SI_Reference_Point/TTL/si.ttl
    * Quantities and Units
        * doc: https://www.omg.org/spec/Commons
        * serialization: https://www.omg.org/spec/Commons/QuantitiesAndUnits.ttl

Use this function to record execution times.

In [7]:
def now() -> str:
    """
    Returns the current time in ISO 8601 format with UTC timezone in the following format:
    YYYY-MM-DDTHH:MM:SS.sssZ
    """
    timestamp = datetime.datetime.now(datetime.timezone.utc)
    timestamp_formated = timestamp.strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3]  +"Z"

    return timestamp_formated

Register yourself in the Knowledge Graph using ProvO. Change the given name, family name and immatriculation number to reflect your own data.

In [8]:
# Ontologies used: foaf, prov, IAO
reigstration_triples_a = [
f':{student_a} rdf:type foaf:Person .',
f':{student_a} rdf:type prov:Agent .',
f':{student_a} foaf:givenName "Avelardo" .',
f':{student_a} foaf:familyName "Ramirez" .',
f':{student_a} <http://vivoweb.org/ontology/core#identifier> :{student_a} .',
f':{student_a} rdf:type <http://purl.obolibrary.org/obo/IAO_0000578> .',
f':{student_a} <http://www.w3.org/2000/01/rdf-schema#label> "Immatriculation number" .',
f':{student_a} <http://purl.obolibrary.org/obo/IAO_0000219> "12435655"^^xsd:string .',
]

reigstration_triples_b = [
f':{student_b} rdf:type foaf:Person .',
f':{student_b} rdf:type prov:Agent .',
f':{student_b} foaf:givenName "Agon" .',
f':{student_b} foaf:familyName "Sylejmani" .',
f':{student_b} <http://vivoweb.org/ontology/core#identifier> :{student_b} .',
f':{student_b} rdf:type <http://purl.obolibrary.org/obo/IAO_0000578> .',
f':{student_b} <http://www.w3.org/2000/01/rdf-schema#label> "Immatriculation number" .',
f':{student_b} <http://purl.obolibrary.org/obo/IAO_0000219> "01556207"^^xsd:string .',
]

role_triples = [
    f':{code_writer_role} rdf:type prov:Role .',
    f':{code_executor_role} rdf:type prov:Role .',
]


engine.insert(reigstration_triples_a, prefixes=prefixes)
engine.insert(reigstration_triples_b, prefixes=prefixes)
engine.insert(role_triples, prefixes=prefixes)

EndPointInternalError: EndPointInternalError: The endpoint returned the HTTP status code 500. 

Response:
b'Entity pool initialization failure'

**What not do do**

Do not use [blank nodes](https://www.w3.org/wiki/BlankNodes).

PROV-O uses blank nodes to connect multiple elements with each other.
Such blank nodes (such as _:association) should not be used.
Instead, assign a fixed node ID such as
:5119fcd7-b571-41e0-9464-a37c7be0f574 by generating them outside of the
notebook.
We suggest that, for each setting where such a blank node is needed to
connect multiple elements, you create a unique hash (using uuid.uuid4())
and keep this as hard-coded identifier for the blank node. The template
notebook contains examples of this. Do *not* use these provided values,
as otherwise, your provenance documentations will all be connected via
these identifiers!
Also, do not generate them dynamically in every cell execution, e.g. by
using uuid.uuid4() in a cell. This would generate many new linking nodes
for connecting the same elements.
Compute one for each node (cell) where you need them and make sure to
use the same one on each re-execution of the notebook.

In [10]:
# Directory for obesity dataset
obesity_data_path = os.path.join("data", "datasets", "obesity")
os.makedirs(obesity_data_path, exist_ok=True)


## Business Understanding 

In [11]:
## Each Activity that follows is part of the Business Understanding Phase

business_understanding_phase_executor = [
f':business_understanding_phase rdf:type prov:Activity .',
f':business_understanding_phase rdfs:label "Business Understanding Phase" .', ## Phase 1: Business Understanding
]
engine.insert(business_understanding_phase_executor, prefixes=prefixes)


EndPointInternalError: EndPointInternalError: The endpoint returned the HTTP status code 500. 

Response:
b'Entity pool initialization failure'

In [12]:
#############################################
# Documentation - Business Understanding
#############################################

data_src_and_scenario_comment = """
**Data Source:**
The dataset contains 2,111 records from individuals in Mexico, Peru, and Colombia,
collected to estimate obesity levels based on eating habits and physical condition.
The data includes 17 attributes covering demographics (age, gender, height, weight),
eating habits (high-calorie food consumption, vegetable consumption, number of meals,
water intake, alcohol consumption), and physical activity patterns (exercise frequency,
technology usage time, transportation mode).

**Business Scenario:**
A public health agency in Latin America aims to combat the rising obesity epidemic
by implementing targeted intervention programs. The agency needs an automated system
to classify individuals into obesity risk categories based on their lifestyle and
physical characteristics. This classification will enable:
1. Early identification of at-risk populations
2. Personalized health recommendations
3. Resource allocation for intervention programs
4. Monitoring of public health trends over time

The system will be deployed as a web-based screening tool accessible to healthcare
providers and wellness centers across Mexico, Peru, and Colombia.
"""

business_objectives_comment = """
The primary business objectives are:

1. **Reduce Obesity Prevalence:** Support public health initiatives aimed at
   reducing obesity rates.

2. **Enable Targeted Interventions:** Provide healthcare professionals with an
   accurate classification tool that identifies specific obesity risk categories,
   allowing for customized intervention strategies for each risk group.

3. **Improve Resource Allocation:** Help health agencies allocate resources
   efficiently by identifying geographic regions and demographic groups with
   highest obesity risk.

4. **Support Preventive Care:** Enable early detection of obesity risk before
   severe health complications develop.

5. **Provide Data-Driven Insights:** Generate actionable insights about the
   relationship between lifestyle factors and obesity levels to inform public health policy.
"""

business_success_criteria_comment = """
The success of this business initiative will be measured by:

1. **Adoption Rate:** Achieve 70% adoption rate among targeted healthcare
   facilities within the first year of deployment.

2. **Intervention Effectiveness:** Demonstrate that individuals identified as
   high-risk who receive targeted interventions show measurable improvement
   (BMI reduction of at least 2 points).

3. **Cost-Effectiveness:** Reduce overall healthcare costs related to obesity
   complications by 15% over 3 years through early intervention.

4. **User Satisfaction:** Achieve at least 80% satisfaction rating from
   healthcare providers using the tool, measured through user surveys.

5. **Coverage:** Successfully screen at least 50,000 individuals within the
   first year across the three target countries.

6. **Actionability:** Ensure that 90% of high-risk classifications result in
   documented intervention actions by healthcare providers.
"""

data_mining_goals_comment = """
The specific data mining goals are:

1. **Multi-class Classification:** Build a robust classifier that accurately
   predicts obesity levels across all 7 categories:
   - Insufficient Weight
   - Normal Weight
   - Overweight Level I
   - Overweight Level II
   - Obesity Type I
   - Obesity Type II
   - Obesity Type III

2. **Feature Importance Analysis:** Identify which eating habits and physical
   activity factors are most predictive of obesity levels to guide intervention
   design.

3. **Balanced Performance:** Achieve strong performance across all obesity
   categories, not just the majority classes, ensuring reliable predictions
   for minority obesity types.

4. **Generalization:** Develop a model that generalizes well across different
   demographic groups (age ranges, genders) and geographic regions.

5. **Interpretability:** Create a model whose predictions can be explained to
   healthcare providers and patients, supporting trust and actionable insights.
"""

data_mining_success_criteria_comment = """
The technical success criteria for the machine learning model are:

1. **Overall Accuracy:** Achieve at least 90% overall classification accuracy
   on held-out test data.

2. **Balanced Performance:**
   - Macro-averaged F1-score ≥ 0.85
   - Minimum per-class recall ≥ 0.75 for each obesity category
   - Macro-averaged precision ≥ 0.85

3. **Confusion Matrix Analysis:** No obesity category should be systematically
   misclassified as another.

4. **Generalization:** Performance on validation set should be within 5% of
   training set performance (avoid overfitting).

5. **Reproducibility:** All results must be reproducible with documented random
   seeds and preprocessing steps.

6. **Baseline Comparison:** Outperform a simple baseline (random classifier,
   majority class classifier) by at least 60 percentage points.
"""

ai_risk_aspects_comment = """
Several AI risk aspects require consideration:

1. **Health Data Privacy:**
   - Risk: Exposure of sensitive health information (weight, eating habits)
   - Mitigation: Ensure anonymization, secure data handling, GDPR compliance

2. **Bias and Fairness:**
   - Risk: Model may perform differently across genders, age groups, or
     geographic regions, leading to unfair treatment
   - Concern: 77% synthetic data may not accurately represent real-world
     distributions
   - Action: Evaluate model performance separately for different demographic
     subgroups; monitor for systematic bias

3. **Stigmatization:**
   - Risk: Incorrect obesity classification could lead to stigmatization,
     discrimination in insurance/employment
   - Mitigation: Predictions should be treated as screening tools, not
     definitive diagnoses; require human oversight

4. **Over-reliance on Automation:**
   - Risk: Healthcare providers may rely solely on model predictions without
     clinical judgment
   - Mitigation: System should support decision-making, not replace professional
     medical assessment

5. **Limited Generalizability:**
   - Risk: Model trained on Latin American populations may not generalize to
     other regions/cultures with different dietary patterns
   - Action: Clearly document limitations; validate before deployment in new
     regions

6. **Synthetic Data Concerns:**
   - Risk: 77% SMOTE-generated data may introduce artificial patterns not
     present in real populations
   - Action: Carefully evaluate model behavior; compare predictions on real vs.
     synthetic data subsets

7. **Feature Sensitivity:**
   - Risk: Model may learn spurious correlations (e.g., gender stereotypes
     about eating habits)
   - Action: Analyze feature importance; test for protected attribute influence

8. **Intervention Harm:**
   - Risk: False positives could lead to unnecessary interventions; false
     negatives could miss at-risk individuals
   - Mitigation: Establish appropriate confidence thresholds; implement
     human-in-the-loop verification for critical cases
"""

bu_ass_uuid_executor = "bb6a40f9-9d92-4f9f-bbd2-b65ef6a82da2"

business_understanding_executor = [
f':business_understanding rdf:type prov:Activity .',
f':business_understanding sc:isPartOf :business_understanding_phase .',
f':business_understanding prov:qualifiedAssociation :{bu_ass_uuid_executor} .',
f':{bu_ass_uuid_executor} prov:agent :{executed_by} .',
f':{bu_ass_uuid_executor} rdf:type prov:Association .',
f':{bu_ass_uuid_executor} prov:hadRole :{code_executor_role} .',
]
engine.insert(business_understanding_executor, prefixes=prefixes)


business_understanding_data_executor = [
# 1a
f':bu_data_source_and_scenario rdf:type prov:Entity .',
f':bu_data_source_and_scenario prov:wasGeneratedBy :business_understanding .',
f':bu_data_source_and_scenario rdfs:label "1a Data Source and Scenario" .',
f':bu_data_source_and_scenario rdfs:comment """{data_src_and_scenario_comment}""" .',
# 1b
f':bu_business_objectives rdf:type prov:Entity .',
f':bu_business_objectives prov:wasGeneratedBy :business_understanding .',
f':bu_business_objectives rdfs:label "1b Business Objectives" .',
f':bu_business_objectives rdfs:comment """{business_objectives_comment}""" .',
# 1c
f':bu_business_success_criteria rdf:type prov:Entity .',
f':bu_business_success_criteria prov:wasGeneratedBy :business_understanding .',
f':bu_business_success_criteria rdfs:label "1c Business Success Criteria" .',
f':bu_business_success_criteria rdfs:comment """{business_success_criteria_comment}""" .',
# 1d
f':bu_data_mining_goals rdf:type prov:Entity .',
f':bu_data_mining_goals prov:wasGeneratedBy :business_understanding .',
f':bu_data_mining_goals rdfs:label "1d Data Mining Goals" .',
f':bu_data_mining_goals rdfs:comment """{data_mining_goals_comment}""" .',
# 1e
f':bu_data_mining_success_criteria rdf:type prov:Entity .',
f':bu_data_mining_success_criteria prov:wasGeneratedBy :business_understanding .',
f':bu_data_mining_success_criteria rdfs:label "1e Data Mining Success Criteria" .',
f':bu_data_mining_success_criteria rdfs:comment """{data_mining_success_criteria_comment}""" .',
# 1f
f':bu_ai_risk_aspects rdf:type prov:Entity .',
f':bu_ai_risk_aspects prov:wasGeneratedBy :business_understanding .',
f':bu_ai_risk_aspects rdfs:label "1f AI risk aspects" .',
f':bu_ai_risk_aspects rdfs:comment """{ai_risk_aspects_comment}""" .',

]
engine.insert(business_understanding_data_executor, prefixes=prefixes)

EndPointInternalError: EndPointInternalError: The endpoint returned the HTTP status code 500. 

Response:
b'Entity pool initialization failure'

## Data Understanding

The following pseudo-code & pseudo-documentation may be used as a hint.

In [13]:
## Each Activity that follows is part of the Data Understanding Phase

data_understanding_phase = [
    f':data_understanding_phase rdf:type prov:Activity .',
    f':data_understanding_phase rdfs:label "Data Understanding Phase" .',
]
engine.insert(data_understanding_phase, prefixes=prefixes)

EndPointInternalError: EndPointInternalError: The endpoint returned the HTTP status code 500. 

Response:
b'Entity pool initialization failure'

In [14]:
##############################################
# Basic Information (2a) - Loading the data and Analyzing
##############################################

import pandas as pd
import os

obesity_data_path = os.path.join("data", "datasets", "obesity")
os.makedirs(obesity_data_path, exist_ok=True)

# Capture start time
start_time_load = now()

# Load the data
df = pd.read_csv(os.path.join(obesity_data_path, "obesity_data.csv"))

# Capture end time
end_time_load = now()

print(f"Dataset Shape: {df.shape}")
print(f"\nColumn Names:\n{df.columns.tolist()}")
print(f"\nFirst few rows:")
display(df.head())

print(f"\nColumn Names and Types:\n{df.dtypes}")
print(f"\nMissing Values:\n{df.isnull().sum()}")
print(f"\nStatistical Summary:\n{df.describe()}")

# Numeric and categorical columns
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
print(f"\nNumeric features ({len(numeric_cols)}): {numeric_cols}")
print(f"\nCategorical features ({len(categorical_cols)}): {categorical_cols}")

# Data loading documentation will be included in comprehensive activity at end

Dataset Shape: (2111, 17)

Column Names:
['Age', 'Gender', 'Height', 'Weight', 'CALC', 'FAVC', 'FCVC', 'NCP', 'SCC', 'SMOKE', 'CH2O', 'family_history_with_overweight', 'FAF', 'TUE', 'CAEC', 'MTRANS', 'NObeyesdad']

First few rows:


,Age,Gender,Height,Weight,CALC,FAVC,FCVC,NCP,SCC,SMOKE,CH2O,family_history_with_overweight,FAF,TUE,CAEC,MTRANS,NObeyesdad
0,21.0,Female,1.62,64.0,no,no,2.0,3.0,no,no,2.0,yes,0.0,1.0,Sometimes,Public_Transportation,Normal_Weight
1,21.0,Female,1.52,56.0,Sometimes,no,3.0,3.0,yes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,23.0,Male,1.80,77.0,Frequently,no,2.0,3.0,no,no,2.0,yes,2.0,1.0,Sometimes,Public_Transportation,Normal_Weight
3,27.0,Male,1.80,87.0,Frequently,no,3.0,3.0,no,no,2.0,no,2.0,0.0,Sometimes,Walking,Overweight_Level_I
4,22.0,Male,1.78,89.8,Sometimes,no,2.0,1.0,no,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II



Column Names and Types:
Age                               float64
Gender                             object
Height                            float64
Weight                            float64
CALC                               object
FAVC                               object
FCVC                              float64
NCP                               float64
SCC                                object
SMOKE                              object
CH2O                              float64
family_history_with_overweight     object
FAF                               float64
TUE                               float64
CAEC                               object
MTRANS                             object
NObeyesdad                         object
dtype: object

Missing Values:
Age                               0
Gender                            0
Height                            0
Weight                            0
CALC                              0
FAVC                              0
FCVC          

In [15]:
##############################################
# PROVENANCE: Task 2a - Load and Analyze Attributes
##############################################

ld_uuid_exec = "b8bac193-c4e6-4e31-9134-b23e001e279f"
engine.insert([
    f':load_data prov:qualifiedAssociation :{ld_uuid_exec} .',
    f':{ld_uuid_exec} prov:agent :{executed_by} .',
    f':{ld_uuid_exec} rdf:type prov:Association .',
    f':{ld_uuid_exec} prov:hadRole :{code_executor_role} .'
], prefixes=prefixes)

ld_uuid_writer = "c600e15c-87a9-4e2a-be85-b6c2a3014213"
ld_report = "Load Obesity dataset and initial inspection."

engine.insert([
    ':load_data rdf:type prov:Activity .',
    ':load_data sc:isPartOf :data_understanding_phase .',
    ':load_data rdfs:label "Load Obesity Data" .',
    f':load_data rdfs:comment """{ld_report}""" .',
    f':load_data prov:startedAtTime "{start_time_load}"^^xsd:dateTime .',
    f':load_data prov:endedAtTime "{end_time_load}"^^xsd:dateTime .',
    f':load_data prov:qualifiedAssociation :{ld_uuid_writer} .',
    f':{ld_uuid_writer} prov:agent :{student_a} .',
    f':{ld_uuid_writer} rdf:type prov:Association .',
    f':{ld_uuid_writer} prov:hadRole :{code_writer_role} .',
    ':load_data prov:used :raw_data .',
    ':data rdf:type prov:Entity .',
    ':data prov:wasGeneratedBy :load_data .',
    ':data prov:wasDerivedFrom :raw_data .'
], prefixes=prefixes)

engine.insert([
    ':raw_data rdf:type sc:Dataset .',
    ':raw_data rdfs:label "Obesity Raw Dataset" .',
    ':obesity_csv rdf:type cr:FileObject .',
    ':obesity_csv sc:name "obesity_data.csv" .',
    ':obesity_csv sc:encodingFormat "text/csv" .',
    ':raw_data sc:distribution :obesity_csv .',
    ':raw_data cr:recordSet :raw_recordset .',
    ':raw_recordset rdf:type cr:RecordSet .',
    ':raw_recordset cr:field :field_age .',
    ':raw_recordset cr:field :field_gender .',
    ':raw_recordset cr:field :field_weight .',
    ':raw_recordset cr:field :field_target .',
    ':field_age rdf:type cr:Field .',
    ':field_age sc:name "Age" .',
    ':field_age cr:dataType xsd:float .',
    ':field_gender rdf:type cr:Field .',
    ':field_gender sc:name "Gender" .',
    ':field_gender cr:dataType xsd:string .',
    ':field_weight rdf:type cr:Field .',
    ':field_weight sc:name "Weight" .',
    ':field_weight cr:dataType xsd:float .',
    ':field_target rdf:type cr:Field .',
    ':field_target sc:name "NObeyesdad" .',
    ':field_target cr:dataType xsd:string .'
], prefixes=prefixes)

EndPointInternalError: EndPointInternalError: The endpoint returned the HTTP status code 500. 

Response:
b'Entity pool initialization failure'

In [16]:
# Documenting the dataset using Croissant
raw_data_description = [
    ':data sc:name "Obesity Levels Dataset" .',
    ':data sc:description "Dataset containing obesity levels based on eating habits and physical condition from individuals in Mexico, Peru, and Colombia. Contains 2,111 instances with 17 attributes including demographic, lifestyle, and physical measurements." .',

    # Record set
    ':obesity_recordset rdf:type cr:RecordSet .',
    ':obesity_recordset sc:name "Obesity data records" .',
    ':data cr:recordSet :obesity_recordset .',

    # NUMERIC FIELDS

    ':field_age rdf:type cr:Field .',
    ':field_age sc:name "Age" .',
    ':field_age sc:description "Age of the individual in years" .',
    ':field_age cr:dataType xsd:integer .',
    ':field_age qudt:unit siu:year .',
    ':obesity_recordset cr:field :field_age .',

    ':field_height rdf:type cr:Field .',
    ':field_height sc:name "Height" .',
    ':field_height sc:description "Height of the individual in meters" .',
    ':field_height cr:dataType xsd:double .',
    ':field_height qudt:unit siu:metre .',
    ':obesity_recordset cr:field :field_height .',

    ':field_weight rdf:type cr:Field .',
    ':field_weight sc:name "Weight" .',
    ':field_weight sc:description "Weight of the individual in kilograms" .',
    ':field_weight cr:dataType xsd:double .',
    ':field_weight qudt:unit siu:kilogram .',
    ':obesity_recordset cr:field :field_weight .',

    ':field_fcvc rdf:type cr:Field .',
    ':field_fcvc sc:name "FCVC" .',
    ':field_fcvc sc:description "Frequency of vegetable consumption (1-3 scale, where 1=never, 2=sometimes, 3=always)" .',
    ':field_fcvc cr:dataType xsd:double .',
    ':obesity_recordset cr:field :field_fcvc .',

    ':field_ncp rdf:type cr:Field .',
    ':field_ncp sc:name "NCP" .',
    ':field_ncp sc:description "Number of main meals consumed per day (typically 1-4)" .',
    ':field_ncp cr:dataType xsd:double .',
    ':field_ncp qudt:unit qudt:CountingUnit .',
    ':obesity_recordset cr:field :field_ncp .',

    ':field_ch2o rdf:type cr:Field .',
    ':field_ch2o sc:name "CH2O" .',
    ':field_ch2o sc:description "Daily water consumption in liters" .',
    ':field_ch2o cr:dataType xsd:double .',
    ':field_ch2o qudt:unit siu:litre .',
    ':obesity_recordset cr:field :field_ch2o .',

    ':field_faf rdf:type cr:Field .',
    ':field_faf sc:name "FAF" .',
    ':field_faf sc:description "Physical activity frequency per week (0-3 scale, where 0=no activity, 3=4+ days/week)" .',
    ':field_faf cr:dataType xsd:double .',
    ':obesity_recordset cr:field :field_faf .',

    ':field_tue rdf:type cr:Field .',
    ':field_tue sc:name "TUE" .',
    ':field_tue sc:description "Time using technology devices (computer, smartphone, TV, etc.) in hours per day" .',
    ':field_tue cr:dataType xsd:double .',
    ':field_tue qudt:unit siu:hour .',
    ':obesity_recordset cr:field :field_tue .',

    # CATEGORICAL FIELDS (9 total)

    # Gender
    ':field_gender rdf:type cr:Field .',
    ':field_gender sc:name "Gender" .',
    ':field_gender sc:description "Gender of the individual (Female/Male)" .',
    ':field_gender cr:dataType xsd:string .',
    ':obesity_recordset cr:field :field_gender .',

    # Family history with overweight
    ':field_family_history rdf:type cr:Field .',
    ':field_family_history sc:name "family_history_with_overweight" .',
    ':field_family_history sc:description "Whether the individual has family members with overweight (yes/no)" .',
    ':field_family_history cr:dataType xsd:string .',
    ':obesity_recordset cr:field :field_family_history .',

    # FAVC - Frequent consumption of high caloric food
    ':field_favc rdf:type cr:Field .',
    ':field_favc sc:name "FAVC" .',
    ':field_favc sc:description "Frequent consumption of high caloric food (yes/no)" .',
    ':field_favc cr:dataType xsd:string .',
    ':obesity_recordset cr:field :field_favc .',

    # CAEC - Consumption of food between meals
    ':field_caec rdf:type cr:Field .',
    ':field_caec sc:name "CAEC" .',
    ':field_caec sc:description "Consumption of food between meals (no/Sometimes/Frequently/Always)" .',
    ':field_caec cr:dataType xsd:string .',
    ':obesity_recordset cr:field :field_caec .',

    # SMOKE - Smoking habit
    ':field_smoke rdf:type cr:Field .',
    ':field_smoke sc:name "SMOKE" .',
    ':field_smoke sc:description "Whether the individual smokes (yes/no)" .',
    ':field_smoke cr:dataType xsd:string .',
    ':obesity_recordset cr:field :field_smoke .',

    # SCC - Calorie consumption monitoring
    ':field_scc rdf:type cr:Field .',
    ':field_scc sc:name "SCC" .',
    ':field_scc sc:description "Monitors calorie consumption (yes/no)" .',
    ':field_scc cr:dataType xsd:string .',
    ':obesity_recordset cr:field :field_scc .',

    # CALC - Alcohol consumption
    ':field_calc rdf:type cr:Field .',
    ':field_calc sc:name "CALC" .',
    ':field_calc sc:description "Frequency of alcohol consumption (no/Sometimes/Frequently/Always)" .',
    ':field_calc cr:dataType xsd:string .',
    ':obesity_recordset cr:field :field_calc .',

    # MTRANS - Mode of transportation
    ':field_mtrans rdf:type cr:Field .',
    ':field_mtrans sc:name "MTRANS" .',
    ':field_mtrans sc:description "Mode of transportation usually used (Automobile/Motorbike/Bike/Public_Transportation/Walking)" .',
    ':field_mtrans cr:dataType xsd:string .',
    ':obesity_recordset cr:field :field_mtrans .',

    # NObeyesdad - Target variable (Obesity level)
    ':field_nobeyesdad rdf:type cr:Field .',
    ':field_nobeyesdad sc:name "NObeyesdad" .',
    ':field_nobeyesdad sc:description "Obesity level classification: Insufficient_Weight, Normal_Weight, Overweight_Level_I, Overweight_Level_II, Obesity_Type_I, Obesity_Type_II, Obesity_Type_III" .',
    ':field_nobeyesdad cr:dataType xsd:string .',
    ':obesity_recordset cr:field :field_nobeyesdad .',
]

engine.insert(raw_data_description, prefixes=prefixes)

EndPointInternalError: EndPointInternalError: The endpoint returned the HTTP status code 500. 

Response:
b'Entity pool initialization failure'

In [17]:
##############################################
# Statistical Properties (2b)
##############################################

start_time_stats = now()

print("STATISTICAL PROPERTIES AND CORRELATIONS")

# Numeric features
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

print(f"\nDESCRIPTIVE STATISTICS (Numeric Features):")
print(df[numeric_cols].describe())

print(f"\nCLASS DISTRIBUTION (Target Variable):")
class_dist = df['NObeyesdad'].value_counts().sort_index()
print(class_dist)
print(f"\nClass Proportions (%):")
print((class_dist / len(df) * 100).round(2))

# Correlation analysis
correlation_matrix = df[numeric_cols].corr()
print(f"\nCORRELATION MATRIX (Numeric Features):")
print(correlation_matrix.round(3))

# Skewness
print(f"\nSKEWNESS (Numeric Features):")
for col in numeric_cols:
    skew = df[col].skew()
    print(f"   {col}: {skew:.3f} {'(right-skewed)' if skew > 0.5 else '(left-skewed)' if skew < -0.5 else '(approximately symmetric)'}")

end_time_stats = now()

STATISTICAL PROPERTIES AND CORRELATIONS

DESCRIPTIVE STATISTICS (Numeric Features):
               Age       Height       Weight         FCVC          NCP  \
count  2111.000000  2111.000000  2111.000000  2111.000000  2111.000000   
mean     24.312600     1.701677    86.586058     2.419043     2.685628   
std       6.345968     0.093305    26.191172     0.533927     0.778039   
min      14.000000     1.450000    39.000000     1.000000     1.000000   
25%      19.947192     1.630000    65.473343     2.000000     2.658738   
50%      22.777890     1.700499    83.000000     2.385502     3.000000   
75%      26.000000     1.768464   107.430682     3.000000     3.000000   
max      61.000000     1.980000   173.000000     3.000000     4.000000   

              CH2O          FAF          TUE  
count  2111.000000  2111.000000  2111.000000  
mean      2.008011     1.010298     0.657866  
std       0.612953     0.850592     0.608927  
min       1.000000     0.000000     0.000000  
25%       1.58

In [18]:
##############################################
# PROVENANCE: Task 2b - Statistical Analysis
##############################################

# CHANGE THESE UUIDs!
t2b_uuid_exec = "22222222-3333-4444-5555-666666666601"
t2b_uuid_writer = "22222222-3333-4444-5555-666666666602"

# Executor
engine.insert([
    f':analyze_statistics prov:qualifiedAssociation :{t2b_uuid_exec} .',
    f':{t2b_uuid_exec} prov:agent :{executed_by} .',
    f':{t2b_uuid_exec} rdf:type prov:Association .',
    f':{t2b_uuid_exec} prov:hadRole :{code_executor_role} .'
], prefixes=prefixes)

# Activity
t2b_code_writer = student_a
t2b_comment = """
Task 2b: Statistical Properties and Correlations
Key Findings:
- Imbalance in class distribution: Obesity_Type_I (25.2%), Normal_Weight (21.5%),
  Overweight_Level_II (13.6%), Overweight_Level_I (13.5%), Obesity_Type_II (13.5%),
  Obesity_Type_III (11.3%), Insufficient_Weight (1.4%)
- No strong correlations (|r| > 0.5) found between numeric features
- Moderate correlations observed:
  * Height-Weight (r=0.463): expected physiological relationship
  * Height-FAF (r=0.295): taller individuals slightly more active
  * Height-NCP (r=0.244): taller individuals eat more meals
- Skewness analysis:
  * Age: 1.529 (right-skewed) - dataset contains more younger individuals
  * NCP: -1.107 (left-skewed) - most people eat 3-4 main meals
  * TUE: 0.619 (right-skewed) - most have low tech use, some high users
  * Other features approximately symmetric
- Descriptive statistics show reasonable ranges for all numeric features
"""

engine.insert([
    ':analyze_statistics rdf:type prov:Activity .',
    ':analyze_statistics sc:isPartOf :data_understanding_phase .',
    ':analyze_statistics rdfs:label "Task 2b: Statistical Properties and Correlations" .',
    f':analyze_statistics rdfs:comment """{t2b_comment}""" .',
    f':analyze_statistics prov:startedAtTime "{start_time_stats}"^^xsd:dateTime .',
    f':analyze_statistics prov:endedAtTime "{end_time_stats}"^^xsd:dateTime .',

    f':analyze_statistics prov:qualifiedAssociation :{t2b_uuid_writer} .',
    f':{t2b_uuid_writer} prov:agent :{t2b_code_writer} .',
    f':{t2b_uuid_writer} rdf:type prov:Association .',
    f':{t2b_uuid_writer} prov:hadRole :{code_writer_role} .',

    # INPUT
    ':analyze_statistics prov:used :data .',

    # OUTPUT
    ':statistical_report rdf:type prov:Entity .',
    ':statistical_report prov:wasGeneratedBy :analyze_statistics .',
    ':statistical_report rdfs:label "Statistical Analysis Report" .',
], prefixes=prefixes)

EndPointInternalError: EndPointInternalError: The endpoint returned the HTTP status code 500. 

Response:
b'Entity pool initialization failure'

In [20]:
##############################################
# Data Quality Analysis (2c)
##############################################

start_time_quality = now()

print("DATA QUALITY ANALYSIS")

# Missing Values
print("\nMISSING VALUES:")
missing_counts = df.isnull().sum()
if missing_counts.sum() > 0:
    missing_pct = (missing_counts / len(df) * 100).round(2)
    missing_df = pd.DataFrame({'Count': missing_counts, 'Percentage': missing_pct})
    print(missing_df[missing_df['Count'] > 0])
else:
    print("No missing values.")

# Duplicates
print(f"\nDUPLICATE ROWS:")
duplicates = df.duplicated().sum()
print(f"   Found {duplicates} duplicate rows ({duplicates/len(df)*100:.2f}%)")

# Outliers Analysis (IQR Method)
print(f"\nOUTLIER DETECTION (IQR Method):")
outlier_info = []
for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = ((df[col] < lower_bound) | (df[col] > upper_bound)).sum()

    outlier_info.append({
        'Feature': col,
        'Outliers': outliers,
        'Percentage': round(outliers/len(df)*100, 2),
        'Lower Bound': round(lower_bound, 2),
        'Upper Bound': round(upper_bound, 2)
    })

outlier_df = pd.DataFrame(outlier_info)
print(outlier_df.to_string(index=False))

# Value Range Plausibility
print(f"\nVALUE PLAUSIBILITY CHECK:")
print(f"Age: Range [{df['Age'].min():.0f}, {df['Age'].max():.0f}] years is plausible")
print(f"Height: Range [{df['Height'].min():.2f}, {df['Height'].max():.2f}]m is plausible")
print(f"Weight: Range [{df['Weight'].min():.1f}, {df['Weight'].max():.1f}]kg is plausible")
print("\nAll values fall within realistic human ranges")

end_time_quality = now()

DATA QUALITY ANALYSIS

MISSING VALUES:
No missing values.

DUPLICATE ROWS:
   Found 24 duplicate rows (1.14%)

OUTLIER DETECTION (IQR Method):
Feature  Outliers  Percentage  Lower Bound  Upper Bound
    Age       168        7.96        10.87        35.08
 Height         1        0.05         1.42         1.98
 Weight         1        0.05         2.54       170.37
   FCVC         0        0.00         0.50         4.50
    NCP       579       27.43         2.15         3.51
   CH2O         0        0.00         0.25         3.82
    FAF         0        0.00        -2.19         3.98
    TUE         0        0.00        -1.50         2.50

VALUE PLAUSIBILITY CHECK:
Age: Range [14, 61] years is plausible
Height: Range [1.45, 1.98]m is plausible
Weight: Range [39.0, 173.0]kg is plausible

All values fall within realistic human ranges


In [21]:
##############################################
# PROVENANCE: Task 2c - Data Quality
##############################################

# CHANGE THESE UUIDs!
t2c_uuid_exec = "33333333-4444-5555-6666-777777777701"
t2c_uuid_writer = "33333333-4444-5555-6666-777777777702"

engine.insert([
    f':assess_data_quality prov:qualifiedAssociation :{t2c_uuid_exec} .',
    f':{t2c_uuid_exec} prov:agent :{executed_by} .',
    f':{t2c_uuid_exec} rdf:type prov:Association .',
    f':{t2c_uuid_exec} prov:hadRole :{code_executor_role} .'
], prefixes=prefixes)

t2c_code_writer = student_a
t2c_comment = """
Task 2c: Data Quality Assessment
Key Findings:
1. Missing Values: None detected
2. Duplicate Rows: 24 duplicates found
3. Outliers (IQR method):
   - Age: 168 outliers (7.96%) - elderly individuals above 35 years
   - NCP: 579 outliers (27.43%) - individuals eating <2.15 or >3.51 meals/day
   - Height: 1 outlier (0.05%) - likely data entry error or very tall individual
   - Weight: 1 outlier (0.05%) - likely very heavy individual or error
   - Other features: no outliers detected
4. Plausibility:
   - Age: [14-61] years ✓ realistic range
   - Height: [1.45-1.98]m ✓ realistic range
   - Weight: [39-173]kg ✓ realistic range
   - All values fall within biologically plausible ranges
5. Categorical Consistency: All categorical variables have expected, consistent values
Data Quality Summary: High quality dataset with minimal issues.
"""
# Serialize outlier findings to JSON for structured storage
outlier_json = outlier_df.to_json(orient='records')

engine.insert([
    ':assess_data_quality rdf:type prov:Activity .',
    ':assess_data_quality sc:isPartOf :data_understanding_phase .',
    ':assess_data_quality rdfs:label "Task 2c: Data Quality Assessment" .',
    f':assess_data_quality rdfs:comment """{t2c_comment}""" .',
    f':assess_data_quality prov:startedAtTime "{start_time_quality}"^^xsd:dateTime .',
    f':assess_data_quality prov:endedAtTime "{end_time_quality}"^^xsd:dateTime .',

    f':assess_data_quality prov:qualifiedAssociation :{t2c_uuid_writer} .',
    f':{t2c_uuid_writer} prov:agent :{t2c_code_writer} .',
    f':{t2c_uuid_writer} rdf:type prov:Association .',
    f':{t2c_uuid_writer} prov:hadRole :{code_writer_role} .',

    # INPUT
    ':assess_data_quality prov:used :data .',

    # OUTPUTS
    ':quality_report rdf:type prov:Entity .',
    ':quality_report prov:wasGeneratedBy :assess_data_quality .',
    ':quality_report rdfs:label "Data Quality Report" .',

    ':outlier_report rdf:type prov:Entity .',
    ':outlier_report prov:wasGeneratedBy :assess_data_quality .',
    ':outlier_report rdfs:label "Outlier Analysis Report" .',
    f':outlier_report rdfs:comment """{outlier_json}""" .',
], prefixes=prefixes)

EndPointInternalError: EndPointInternalError: The endpoint returned the HTTP status code 500. 

Response:
b'Entity pool initialization failure'

In [22]:
##############################################
# Visual Exploration (2d)
##############################################

start_time_viz = now()

import matplotlib.pyplot as plt
import seaborn as sns

print("VISUAL DATA EXPLORATION")

fig, axes = plt.subplots(3, 3, figsize=(15, 12))
fig.suptitle('Obesity Dataset - Visual Exploration', fontsize=16)

# Plot 1: Target distribution
class_counts = df['NObeyesdad'].value_counts().sort_index()
axes[0, 0].bar(range(len(class_counts)), class_counts.values)
axes[0, 0].set_xticks(range(len(class_counts)))
axes[0, 0].set_xticklabels(class_counts.index, rotation=45, ha='right', fontsize=8)
axes[0, 0].set_title('Class Distribution')
axes[0, 0].set_ylabel('Count')

# Plot 2: Age distribution
axes[0, 1].hist(df['Age'], bins=30, edgecolor='black')
axes[0, 1].set_title('Age Distribution')
axes[0, 1].set_xlabel('Age (years)')
axes[0, 1].set_ylabel('Frequency')

# Plot 3: Height vs Weight scatter
axes[0, 2].scatter(df['Height'], df['Weight'], alpha=0.5)
axes[0, 2].set_title('Height vs Weight')
axes[0, 2].set_xlabel('Height (m)')
axes[0, 2].set_ylabel('Weight (kg)')

# Plot 4: Gender distribution
gender_counts = df['Gender'].value_counts()
axes[1, 0].pie(gender_counts, labels=gender_counts.index, autopct='%1.1f%%')
axes[1, 0].set_title('Gender Distribution')

# Plot 5: Correlation heatmap
corr = df[numeric_cols].corr()
sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm', ax=axes[1, 1],
            cbar_kws={'shrink': 0.8})
axes[1, 1].set_title('Feature Correlations')

# Plot 6: Physical activity frequency
axes[1, 2].hist(df['FAF'], bins=20, edgecolor='black')
axes[1, 2].set_title('Physical Activity Frequency')
axes[1, 2].set_xlabel('FAF (0-3 scale)')
axes[1, 2].set_ylabel('Frequency')

# Plot 7: Water consumption
axes[2, 0].hist(df['CH2O'], bins=20, edgecolor='black')
axes[2, 0].set_title('Daily Water Consumption')
axes[2, 0].set_xlabel('Liters per day')
axes[2, 0].set_ylabel('Frequency')

# Plot 8: Box plot of Weight by Obesity Level
df.boxplot(column='Weight', by='NObeyesdad', ax=axes[2, 1])
axes[2, 1].set_title('Weight Distribution by Obesity Level')
axes[2, 1].set_xlabel('Obesity Level')
axes[2, 1].set_ylabel('Weight (kg)')
axes[2, 1].get_figure().suptitle('')  # Remove the automatic title

# Plot 9: Technology use time
axes[2, 2].hist(df['TUE'], bins=20, edgecolor='black')
axes[2, 2].set_title('Technology Use Time')
axes[2, 2].set_xlabel('Hours per day')
axes[2, 2].set_ylabel('Frequency')

plt.tight_layout()

# Save figure
viz_path = os.path.join(obesity_data_path, "data_exploration.png")
plt.savefig(viz_path, dpi=300, bbox_inches='tight')
print(f"\nVisualizations saved to: {viz_path}")
plt.close()

end_time_viz = now()


VISUAL DATA EXPLORATION

Visualizations saved to: data/datasets/obesity/data_exploration.png


In [23]:
##############################################
# PROVENANCE: Task 2d - Visual Exploration
##############################################

# CHANGE THESE UUIDs!
t2d_uuid_exec = "44444444-5555-6666-7777-888888888801"
t2d_uuid_writer = "44444444-5555-6666-7777-888888888802"

engine.insert([
    f':explore_visually prov:qualifiedAssociation :{t2d_uuid_exec} .',
    f':{t2d_uuid_exec} prov:agent :{executed_by} .',
    f':{t2d_uuid_exec} rdf:type prov:Association .',
    f':{t2d_uuid_exec} prov:hadRole :{code_executor_role} .'
], prefixes=prefixes)

t2d_code_writer = student_b
t2d_comment = """
Task 2d: Visual Data Exploration
Created comprehensive visualization dashboard with 9 plots:
1. Class Distribution: Confirms class imbalance identified in 2b
2. Age Distribution: Shows right-skewed pattern, most individuals 20-30 years old
3. Height vs Weight: Positive correlation visible, expected physiological relationship
4. Gender Distribution: Approximately balanced (Female: 51%, Male: 49%)
5. Correlation Heatmap: Visual confirmation of weak correlations between features
6. Physical Activity Frequency: Bimodal distribution - sedentary and active groups
7. Water Consumption: Centered around 2 liters/day with normal distribution
8. Weight by Obesity Level: Clear separation between classes, validating target variable
9. Technology Use Time: Right-skewed, most people use <1 hour/day
Visualization saved to: data/datasets/obesity/data_exploration.png
"""

engine.insert([
    ':explore_visually rdf:type prov:Activity .',
    ':explore_visually sc:isPartOf :data_understanding_phase .',
    ':explore_visually rdfs:label "Task 2d: Visual Exploration" .',
    f':explore_visually rdfs:comment """{t2d_comment}""" .',
    f':explore_visually prov:startedAtTime "{start_time_viz}"^^xsd:dateTime .',
    f':explore_visually prov:endedAtTime "{end_time_viz}"^^xsd:dateTime .',

    f':explore_visually prov:qualifiedAssociation :{t2d_uuid_writer} .',
    f':{t2d_uuid_writer} prov:agent :{t2d_code_writer} .',
    f':{t2d_uuid_writer} rdf:type prov:Association .',
    f':{t2d_uuid_writer} prov:hadRole :{code_writer_role} .',

    # INPUT
    ':explore_visually prov:used :data .',

    # OUTPUTS
    ':visualization_report rdf:type prov:Entity .',
    ':visualization_report prov:wasGeneratedBy :explore_visually .',
    ':visualization_report rdfs:label "Visual Exploration Dashboard" .',
    f':visualization_report sc:contentUrl "file://{viz_path}" .',
], prefixes=prefixes)

EndPointInternalError: EndPointInternalError: The endpoint returned the HTTP status code 500. 

Response:
b'Entity pool initialization failure'

In [24]:
# ==========================================================================
# TASK 2e: ETHICAL SENSITIVITY ASSESSMENT (Manual logging)
# ==========================================================================

# CHANGE THIS UUID!
t2e_uuid_exec = "55555555-6666-7777-8888-999999999901"

# This is a manual analysis, no code execution
t2e_code_writer = student_a
t2e_comment = """
Task 2e: Ethical Sensitivity Assessment
Potentially Sensitive Attributes Identified:
1. Gender (Female/Male)
   - Protected characteristic under most anti-discrimination laws
   - Risk: Model could learn gender stereotypes about eating habits or body composition
   - Mitigation: Ensure equal performance across genders, test for disparate impact

2. Age (14-61 years range)
   - Protected in employment and some health contexts
   - Risk: Age-based discrimination in health interventions
   - Younger individuals (14-17) are minors,this requires special consideration

3. Family History with Overweight (yes/no)
   - Potentially sensitive genetic/family information
   - Risk: Could be used to discriminate based on genetic predisposition
   - Not typically protected but ethically sensitive

Underrepresented Groups:
1. Insufficient Weight class: Only 1.4% of dataset
   - Risk: Model may perform poorly on this minority class
   - Action: Consider oversampling or stratified evaluation

2. Extreme age groups (14-17, 55+): Underrepresented
   - Risk: Model may not generalize well to these age ranges
   - Action: Ensure test set includes these groups for validation

Class Imbalance Analysis:
- Obesity Type I: 25.2% (largest class)
- Insufficient Weight: 1.4% (smallest class)
- 18x difference between largest and smallest class
- Recommendation: Use macro-averaged metrics and per-class evaluation
"""

engine.insert([
    ':assess_ethical_sensitivity rdf:type prov:Activity .',
    ':assess_ethical_sensitivity sc:isPartOf :data_understanding_phase .',
    ':assess_ethical_sensitivity rdfs:label "Task 2e: Ethical Sensitivity Assessment" .',
    f':assess_ethical_sensitivity rdfs:comment """{t2e_comment}""" .',

    f':assess_ethical_sensitivity prov:qualifiedAssociation :{t2e_uuid_exec} .',
    f':{t2e_uuid_exec} prov:agent :{t2e_code_writer} .',
    f':{t2e_uuid_exec} rdf:type prov:Association .',
    f':{t2e_uuid_exec} prov:hadRole :{code_writer_role} .',

    # INPUT
    ':assess_ethical_sensitivity prov:used :data .',
    ':assess_ethical_sensitivity prov:used :statistical_report .',

    # OUTPUT
    ':ethics_assessment rdf:type prov:Entity .',
    ':ethics_assessment prov:wasGeneratedBy :assess_ethical_sensitivity .',
    ':ethics_assessment rdfs:label "Ethical Sensitivity Assessment" .',
], prefixes=prefixes)

EndPointInternalError: EndPointInternalError: The endpoint returned the HTTP status code 500. 

Response:
b'Entity pool initialization failure'

In [25]:
# ==========================================================================
# TASK 2f: BIAS AND RISK ANALYSIS (Manual logging)
# ==========================================================================

# CHANGE THIS UUID!
t2f_uuid_exec = "66666666-7777-8888-9999-000000000001"

t2f_code_writer = student_a
t2f_comment = """
Task 2f: Potential Risks and Bias Analysis
Data Collection Bias:
1. Geographic Bias: Data only from Mexico, Peru, Colombia
   - Risk: Model may not generalize to other populations/regions
   - Question for expert: "Are eating habits and obesity patterns comparable across
     Latin American countries vs. other regions?"

2. Synthetic Data Concerns: Dataset appears to be 77% synthetic (based on Kaggle description)
   - Risk: Synthetic patterns may not reflect real-world complexity
   - Question for expert: "What generation method was used? Were correlations preserved?"

3. Sampling Bias: How were participants recruited?
   - Question for expert: "Was sampling random? Were certain demographics overrepresented?"

Measurement Bias:
1. Self-reported vs. measured data
   - Question for expert: "Are height/weight measured or self-reported? Self-reporting
     tends to underestimate weight and overestimate height"

2. Cultural interpretation of categorical variables
   - Question for expert: "Do terms like 'frequent' or 'sometimes' mean the same across
     cultures? Are there translation issues?"

Label Quality:
- Question for expert: "How was obesity classification determined? BMI alone or other
  criteria? Who performed the classification?"

Historical Bias:
- Data collection timeframe unknown
- Question for expert: "When was data collected? Have dietary patterns changed since?"

Proxy Discrimination Risks:
- Features like transportation mode (MTRANS) could serve as proxies for socioeconomic status
- Question for expert: "Could certain feature combinations inadvertently encode protected
  characteristics like income or education level?"
"""

engine.insert([
    ':analyze_bias_risks rdf:type prov:Activity .',
    ':analyze_bias_risks sc:isPartOf :data_understanding_phase .',
    ':analyze_bias_risks rdfs:label "Task 2f: Bias and Risk Analysis" .',
    f':analyze_bias_risks rdfs:comment """{t2f_comment}""" .',

    f':analyze_bias_risks prov:qualifiedAssociation :{t2f_uuid_exec} .',
    f':{t2f_uuid_exec} prov:agent :{t2f_code_writer} .',
    f':{t2f_uuid_exec} rdf:type prov:Association .',
    f':{t2f_uuid_exec} prov:hadRole :{code_writer_role} .',

    # INPUTS
    ':analyze_bias_risks prov:used :data .',
    ':analyze_bias_risks prov:used :quality_report .',
    ':analyze_bias_risks prov:used :ethics_assessment .',

    # OUTPUT
    ':bias_risk_report rdf:type prov:Entity .',
    ':bias_risk_report prov:wasGeneratedBy :analyze_bias_risks .',
    ':bias_risk_report rdfs:label "Bias and Risk Analysis Report" .',
], prefixes=prefixes)


EndPointInternalError: EndPointInternalError: The endpoint returned the HTTP status code 500. 

Response:
b'Entity pool initialization failure'

In [26]:
# ==========================================================================
# TASK 2g: DATA PREPARATION PLANNING (Manual logging)
# ==========================================================================

# CHANGE THIS UUID!
t2g_uuid_exec = "77777777-8888-9999-0000-111111111101"

t2g_code_writer = student_b
t2g_comment = """
Task 2g: Required Data Preparation Actions
Based on findings from tasks 2a-2f, the following preparation steps are required:

REQUIRED ACTIONS:
1. Remove Duplicate Rows (from 2c)
   - Action: Drop 24 duplicate rows (1.14% of data)
   - Justification: Duplicates provide no additional information and may bias training

2. Handle Outliers (from 2c)
   - NCP outliers (579 cases, 27.43%): RETAIN - likely genuine eating patterns
   - Age outliers (168 cases, 7.96%): RETAIN - represent elderly population

3. Encode Categorical Variables
   - Binary features (Gender, FAVC, SCC, SMOKE): Label encoding (0/1)
   - Ordinal features (CALC): Ordinal encoding
   - Nominal features (CAEC, MTRANS): One-hot encoding
   - Target (NObeyesdad): Label encoding (0-6) for ordinal obesity levels

4. Feature Scaling (from 2b)
   - Standardize numeric features (Age, Height, Weight, FCVC, NCP, CH2O, FAF, TUE)
   - Justification: Features have different scales (Age: years, Height: meters)
   - Method: StandardScaler to ensure mean=0, std=1

5. Create BMI Feature (derived attribute - from 2a)
   - Formula: BMI = Weight / (Height^2)
   - Justification: BMI is standard obesity measure, may improve interpretability

OPTIONAL CONSIDERATIONS:
- Remove highly correlated features: Not needed (max correlation 0.463)
- Binning: Could bin Age into age groups for interpretability
- Interaction features: Height × Weight interaction
- External data: Dietary guidelines, regional health statistics (out of scope)

"""

engine.insert([
    ':plan_data_preparation rdf:type prov:Activity .',
    ':plan_data_preparation sc:isPartOf :data_understanding_phase .',
    ':plan_data_preparation rdfs:label "Task 2g: Data Preparation Planning" .',
    f':plan_data_preparation rdfs:comment """{t2g_comment}""" .',

    f':plan_data_preparation prov:qualifiedAssociation :{t2g_uuid_exec} .',
    f':{t2g_uuid_exec} prov:agent :{t2g_code_writer} .',
    f':{t2g_uuid_exec} rdf:type prov:Association .',
    f':{t2g_uuid_exec} prov:hadRole :{code_writer_role} .',

    # INPUTS - uses all previous reports
    ':plan_data_preparation prov:used :statistical_report .',
    ':plan_data_preparation prov:used :quality_report .',
    ':plan_data_preparation prov:used :ethics_assessment .',
    ':plan_data_preparation prov:used :bias_risk_report .',

    # OUTPUT
    ':preparation_plan rdf:type prov:Entity .',
    ':preparation_plan prov:wasGeneratedBy :plan_data_preparation .',
    ':preparation_plan rdfs:label "Data Preparation Action Plan" .',
], prefixes=prefixes)

HTTPError: HTTP Error 403: Forbidden

## Data Preparation

In [27]:
## Each Activity that follows is part of the Data Preparation Phase

data_preparation_phase_executor = [
f':data_preparation_phase rdf:type prov:Activity .',
f':data_preparation_phase rdfs:label "Data Preparation Phase" .', 
]
engine.insert(data_preparation_phase_executor, prefixes=prefixes)

EndPointInternalError: EndPointInternalError: The endpoint returned the HTTP status code 500. 

Response:
b'Entity pool initialization failure'

In [28]:
# ##########################################
# 3. DATA PREPARATION (Main Pipeline)
# ##########################################

from sklearn.preprocessing import LabelEncoder, StandardScaler
import pandas as pd

# functions for data preparation

def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    """Removes duplicates. Outliers are retained (see 3b)."""
    # 2g: Remove duplicates
    df = df.drop_duplicates().copy()
    return df

def feature_engineering(df: pd.DataFrame) -> pd.DataFrame:
    """Calculates BMI and bins Age."""
    # 2g: BMI Calculation
    df['BMI'] = df['Weight'] / (df['Height'] ** 2)

    # 2g: Age Binning (0: Youth, 1: YoungAdult, 2: Adult, 3: Senior)
    df['Age_Group'] = pd.cut(df['Age'], bins=[0, 25, 40, 60, 100], labels=[0, 1, 2, 3])
    df['Age_Group'] = df['Age_Group'].astype(int)
    return df

def encode_features(df: pd.DataFrame) -> pd.DataFrame:

    # Target
    le = LabelEncoder()
    df['NObeyesdad'] = le.fit_transform(df['NObeyesdad'])

    # Ordinal features
    ord_map = {'no': 0, 'Sometimes': 1, 'Frequently': 2, 'Always': 3}
    df['CAEC'] = df['CAEC'].map(ord_map)
    df['CALC'] = df['CALC'].map(ord_map)

    # Binary features
    bin_map = {'no': 0, 'yes': 1}
    for c in ['FAVC', 'SCC', 'SMOKE', 'family_history_with_overweight']:
        df[c] = df[c].map(bin_map)

    # Nominal (One-Hot)
    df['Gender'] = df['Gender'].map({'Female': 0, 'Male': 1})
    df = pd.get_dummies(df, columns=['MTRANS'], prefix='MTRANS', dtype=int)
    return df

def scale_features(df: pd.DataFrame) -> pd.DataFrame:

    scaler = StandardScaler()
    num_cols = ['Age', 'Height', 'Weight', 'BMI', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE']
    df[num_cols] = scaler.fit_transform(df[num_cols])
    return df

# execution
dp_code_writer = student_b
dp_code_executor = executed_by

start_time_dp = now()

df = clean_data(df)
df = feature_engineering(df)
df = encode_features(df)
df = scale_features(df)

end_time_dp = now()
print(f"Data Prep Completed. Final Shape: {df.shape}")

#############################################
# Documentation
#############################################
# This is the continuation of the example from the Data Understanding phase above.
# There are three steps involved in this process:
# 1. activity creates a figure, report etc. => already done in data understanding phase
# 2. activity inspects the outcome and derives decisions => already done in data understanding phase
# 3. activity follows up on the decision by changing the data => in this case by removing the the outliers that were found

ro_ass_uuid_executor = "ec7e81e1-86ea-475a-a8d4-c7d8ee535000"

dp_executor = [
    f':prepare_data prov:qualifiedAssociation :{ro_ass_uuid_executor} .',
    f':{ro_ass_uuid_executor} prov:agent :{executed_by} .',
    f':{ro_ass_uuid_executor} rdf:type prov:Association .',
    f':{ro_ass_uuid_executor} prov:hadRole :{code_executor_role} .',
]
try:
    engine.insert(dp_executor, prefixes=prefixes)
except:
    print("Graph Error (Executor)")

# Activity & Report Node (Template UUID)
td_ass_uuid_writer = "1405f15a-3545-4014-a962-637f3c10a000"

td_comment = """
**3a. Applied Pre-processing Actions:**
1. **Cleaning:** Deduplicated dataset (24 duplicates removed).
2. **Feature Engineering:** Calculated 'BMI' and created 'Age_Group' bins.
3. **Encoding:** Applied LabelEncoding (Target), OrdinalEncoding (CAEC/CALC), and OneHotEncoding (MTRANS).
4. **Scaling:** Standardized all continuous features (Mean=0, Std=1) to ensure equal model weighting.
"""

dp_activity = [
    ':prepare_data rdf:type prov:Activity .',
    ':prepare_data sc:isPartOf :data_preparation_phase .',
    ':prepare_data rdfs:label "Data Preparation (Full Pipeline)" .',
    f':prepare_data rdfs:comment """{td_comment}""" .',
    f':prepare_data prov:startedAtTime "{start_time_dp}"^^xsd:dateTime .',
    f':prepare_data prov:endedAtTime "{end_time_dp}"^^xsd:dateTime .',

    f':prepare_data prov:qualifiedAssociation :{td_ass_uuid_writer} .',
    f':{td_ass_uuid_writer} prov:agent :{dp_code_writer} .',
    f':{td_ass_uuid_writer} rdf:type prov:Association .',
    f':{td_ass_uuid_writer} prov:hadRole :{code_writer_role} .',

    ':prepare_data prov:used :data .',
    ':prepare_data prov:used :preparation_plan .',

    ':prepared_data rdf:type prov:Entity .',
    ':prepared_data prov:wasGeneratedBy :prepare_data .',
    ':prepared_data prov:wasDerivedFrom :data .',
    ':prepared_data rdf:type sc:Dataset .'
]

try:
    engine.insert(dp_activity, prefixes=prefixes)
    print("Graph: Main Data Prep logged.")
except Exception as e:
    print(f"Graph Error: {e}")

Data Prep Completed. Final Shape: (2087, 23)
Graph Error (Executor)
Graph Error: EndPointInternalError: The endpoint returned the HTTP status code 500. 

Response:
b'Entity pool initialization failure'


**Continue with other tasks of the Data Preparation phase such as binning, scaling etc...**

In [29]:
#############################################
# Documentation 3b: Steps not applied
#############################################

# report for data inspection
comment_3b = """
Report on Pre-processing Steps Considered but Rejected:

During our Data Preparation phase, we evaluated several common techniques but decided not to apply them for the following reasons:

1.  Outlier Removal (Statistical):
    * Observation: We found many data points in 'Weight' and 'BMI' that are statistically considered outliers (far from the mean).
    * Decision: We kept them (RETAINED).
    * Reason: Our specific goal is to classify obesity types. The "outliers" are actually the people with Obesity Type III. If we remove them to "clean" the data, we would delete the exact class we are trying to predict. In this medical context, extreme values are real data, not noise.

2.  Imputation of Missing Values:
    * Observation: We ran df.isnull().sum() and checked the dataset documentation.
    * Result: The dataset is completely full (0 null values).
    * Decision: No imputation (filling gaps with mean/median) was necessary because the data quality is already perfect in this regard.

3.  Dimensionality Reduction (PCA):
    * Idea: We considered using Principal Component Analysis (PCA) to combine features and make the model faster.
    * Decision: SKIPPED.
    * Reason: Our business goal includes "Interpretability". We need to tell users why they are classified as obese (e.g., "because you don't eat vegetables"). PCA turns features into abstract math (Component 1, Component 2) which cannot be explained to a doctor or patient.
"""

# UUID for the writer of this specific documentation
uuid_3b_writer = "52c3f822-002e-4dff-b2bb-bd1feb076035"

doc_3b_triples = [
        ':document_rejected_steps rdf:type prov:Activity .',
        ':document_rejected_steps sc:isPartOf :data_preparation_phase .',
        ':document_rejected_steps rdfs:label "Task 3b: Document Rejected Steps" .',

        f':document_rejected_steps prov:qualifiedAssociation :{uuid_3b_writer} .',
        f':{uuid_3b_writer} prov:agent :{dp_code_writer} .',
        f':{uuid_3b_writer} rdf:type prov:Association .',
        f':{uuid_3b_writer} prov:hadRole :{code_writer_role} .',

        ':data_prep_not_applied rdf:type prov:Entity .',
        ':data_prep_not_applied prov:wasGeneratedBy :document_rejected_steps .',
        ':data_prep_not_applied rdfs:label "3b Steps considered but not applied" .',
        f':data_prep_not_applied rdfs:comment """{comment_3b}""" .',

]

try:
    engine.insert(doc_3b_triples, prefixes=prefixes)
    print("Graph update: 3b (Detailed Report with UUID) logged.")
except Exception as e:
    print(f"Server error: {e}")

Server error: HTTP Error 403: Forbidden


In [30]:
#############################################
# Documentation 3b: Steps not applied
#############################################

comment_3b = """
Report on Pre-processing Steps Considered but Rejected:

During our Data Preparation phase, we evaluated several common techniques but decided not to apply them for the following reasons:

1.  Outlier Removal (Statistical):
    * Observation: We found many data points in 'Weight' and 'BMI' that are statistically considered outliers (far from the mean).
    * Decision: We kept them (RETAINED).
    * Reason: Our specific goal is to classify obesity types. The "outliers" are actually the people with Obesity Type III. If we remove them to "clean" the data, we would delete the exact class we are trying to predict. In this medical context, extreme values are real data, not noise.

2.  Imputation of Missing Values:
    * Observation: We ran df.isnull().sum() and checked the dataset documentation.
    * Result: The dataset is completely full (0 null values).
    * Decision: No imputation (filling gaps with mean/median) was necessary because the data quality is already perfect in this regard.

3.  Dimensionality Reduction (PCA):
    * Idea: We considered using Principal Component Analysis (PCA) to combine features and make the model faster.
    * Decision: SKIPPED.
    * Reason: Our business goal includes "Interpretability". We need to tell users why they are classified as obese (e.g., "because you don't eat vegetables"). PCA turns features into abstract math (Component 1, Component 2) which cannot be explained to a doctor or patient.
"""

# UUID for the writer
uuid_3b_writer = "feee33de-d60c-4f0a-934b-628d946a1256"

# Check if exists first
try:
    check_3b = f"""
    {prefix_header}
    SELECT ?type WHERE {{
        :data_prep_not_applied rdf:type ?type .
    }}
    LIMIT 1
    """
    result = engine.query(check_3b)

    if not result.empty:
        print("Task 3b already exists - skipping")
        task_3b_exists = True
    else:
        print("Task 3b doesn't exist - safe to insert")
        task_3b_exists = False
except Exception as e:
    print(f"Check failed: {e} - assuming doesn't exist")
    task_3b_exists = False

# Only insert if doesn't exist
if not task_3b_exists:
    doc_3b_triples = [
        ':document_rejected_steps rdf:type prov:Activity .',
        ':document_rejected_steps sc:isPartOf :data_preparation_phase .',
        ':document_rejected_steps rdfs:label "Task 3b: Document Rejected Steps" .',

        f':document_rejected_steps prov:qualifiedAssociation :{uuid_3b_writer} .',
        f':{uuid_3b_writer} prov:agent :{dp_code_writer} .',
        f':{uuid_3b_writer} rdf:type prov:Association .',
        f':{uuid_3b_writer} prov:hadRole :{code_writer_role} .',

        ':data_prep_not_applied rdf:type prov:Entity .',
        ':data_prep_not_applied prov:wasGeneratedBy :document_rejected_steps .',
        ':data_prep_not_applied rdfs:label "3b Steps considered but not applied" .',
        f':data_prep_not_applied rdfs:comment """{comment_3b}""" .',
    ]

    try:
        engine.insert(doc_3b_triples, prefixes=prefixes)
        print("Task 3b logged successfully")
    except Exception as e:
        print(f"Insert error: {e}")
else:
    print("⚠️ Skipping Task 3b insert - already exists")

Check failed: EndPointInternalError: The endpoint returned the HTTP status code 500. 

Response:
b'Entity pool initialization failure' - assuming doesn't exist
Insert error: HTTP Error 403: Forbidden


In [32]:
# Check if Task 3b already exists (CORRECT VERSION)
try:
    check_3b = f"""
    {prefix_header}
    SELECT ?type WHERE {{
        :data_prep_not_applied rdf:type ?type .
    }}
    LIMIT 1
    """
    result = engine.query(check_3b)

    if not result.empty:
        print("⚠️ Task 3b (:data_prep_not_applied) already exists!")
        print("Skipping to avoid 403")
        task_3b_exists = True
    else:
        print("✅ Task 3b doesn't exist - safe to insert")
        task_3b_exists = False

except Exception as e:
    print(f"⚠️ Check failed: {e}")
    print("Assuming doesn't exist - will try to insert")
    task_3b_exists = False

# Then use task_3b_exists to decide whether to insert
if not task_3b_exists:
    # Your insert code here
    doc_3b_triples = [...]
    try:
        engine.insert(doc_3b_triples, prefixes=prefixes)
        print("✅ Task 3b logged")
    except Exception as e:
        print(f"❌ Error: {e}")
else:
    print("⚠️ Skipping Task 3b insert")

⚠️ Check failed: EndPointInternalError: The endpoint returned the HTTP status code 500. 

Response:
b'Entity pool initialization failure'
Assuming doesn't exist - will try to insert
❌ Error: 'ellipsis' object is not subscriptable


In [33]:
#############################################
# Documentation 3c: Derived Attributes
#############################################

# Detailed for Feature Engineering
comment_3c = """
Analysis of Derived Attributes (Feature Engineering):

We analyzed which new features could help the model learn better patterns:

1.  BMI (Body Mass Index) - [APPLIED]:
    * We calculated this using Weight / (Height^2).
    * Why: Even though the model has Height and Weight, providing the explicit BMI ratio helps decision trees make cleaner splits. It is historically the strongest predictor for obesity.

2.  Age Grouping (Binning) - [APPLIED]:
    * We converted the continuous 'Age' into categories (Youth, Adult, Senior).
    * Why: Lifestyle habits change with life stages. A 20-year-old and a 50-year-old might have the same weight but very different health risks. This helps the model find those non-linear patterns.

3.  "Sedentary Ratio" (TUE / FAF) - [REJECTED]:
    * Idea: Create a ratio of "Time on Technology" divided by "Physical Activity" to quantify a sedentary lifestyle.
    * Problem: Many participants have FAF = 0 (no activity). This causes division-by-zero errors. Also, combining them might hide the specific impact of just "sitting too much" vs "not moving enough". We kept them separate.

4.  Healthy Diet Score - [REJECTED]:
    * Idea: Summing up vegetable intake and water, subtracting junk food.
    * Reason: Information Loss. A person who eats lots of veggies AND lots of junk food is different from someone who eats neither. The model needs to see the individual habits to classify correctly.
"""

# UUID for the writer of this specific documentation
uuid_3c_writer = "73c922e3-87d2-5c9b-03f2-b2c3d4e5f6g7"

doc_3c_triples = [
    # Activity
    ':analyze_derived_attributes rdf:type prov:Activity .',
    ':analyze_derived_attributes sc:isPartOf :data_preparation_phase .',
    ':analyze_derived_attributes rdfs:label "Task 3c: Derived Attributes Analysis" .',

    # Association with UUID
    f':analyze_derived_attributes prov:qualifiedAssociation :{uuid_3c_writer} .',
    f':{uuid_3c_writer} prov:agent :{dp_code_writer} .',
    f':{uuid_3c_writer} rdf:type prov:Association .',
    f':{uuid_3c_writer} prov:hadRole :{code_writer_role} .',

    # Entity
    ':data_prep_derived_attrs rdf:type prov:Entity .',
    ':data_prep_derived_attrs prov:wasGeneratedBy :analyze_derived_attributes .',
    ':data_prep_derived_attrs rdfs:label "3c Derived Attributes Analysis" .',
    f':data_prep_derived_attrs rdfs:comment """{comment_3c}""" .',
]

try:
    engine.insert(doc_3c_triples, prefixes=prefixes)
    print("Graph update: 3c (Detailed Report with UUID) logged.")
except Exception as e:
    print(f"Server error: {e}")

Server error: EndPointInternalError: The endpoint returned the HTTP status code 500. 

Response:
b'Entity pool initialization failure'


In [34]:
#############################################
# Documentation 3d: External Data
#############################################

# report for hypothetical data
comment_3d = """
Analysis of Additional External Data Sources:

The current dataset is limited to self-reported surveys. We identified three external sources that would significantly improve model quality in a real-world project:

1.  Objective Wearable Data (IoT):
    * The Problem: People lie on surveys. They overestimate how much they run and underestimate how much they sit.
    * The Solution: Integrating data from Fitbits or Apple Health (Step count, Heart Rate Variability, Active Energy Burn).
    * Impact: This would replace subjective "feelings" about activity with hard facts, drastically reducing noise in the FAF (Physical Activity) feature.

2.  Socio-Economic & Location Data:
    * The Context: Obesity is often strongly correlated with income and location (access to healthy food).
    * The Data: Linking user Zip Codes to average household income or "Food Desert" maps.
    * Impact: This would help the model understand if someone eats junk food (FAVC) by choice or because fresh produce isn't available in their area.

3.  Medical & Genetic History:
    * The Missing Link: The dataset assumes weight is 100% lifestyle. It ignores metabolism.
    * The Data: Thyroid function tests, hormone levels, or genetic markers.
    * Impact: This would identify patients who are obese due to medical conditions, not just diet. Currently, the model might unfairly classify these patients based on their "average" diet.
"""

# UUID for the writer of this specific documentation
uuid_3d_writer = "83d033f4-98e3-6d0c-14g3-c3d4e5f6g7h8"

doc_3d_triples = [
    # Activity
    ':analyze_external_data rdf:type prov:Activity .',
    ':analyze_external_data sc:isPartOf :data_preparation_phase .',
    ':analyze_external_data rdfs:label "Task 3d: External Data Analysis" .',

    # Association with UUID
    f':analyze_external_data prov:qualifiedAssociation :{uuid_3d_writer} .',
    f':{uuid_3d_writer} prov:agent :{dp_code_writer} .',
    f':{uuid_3d_writer} rdf:type prov:Association .',
    f':{uuid_3d_writer} prov:hadRole :{code_writer_role} .',

    # Entity
    ':data_prep_external_data rdf:type prov:Entity .',
    ':data_prep_external_data prov:wasGeneratedBy :analyze_external_data .',
    ':data_prep_external_data rdfs:label "3d External Data Analysis" .',
    f':data_prep_external_data rdfs:comment """{comment_3d}""" .',
]

try:
    engine.insert(doc_3d_triples, prefixes=prefixes)
    print("Graph update: 3d (Detailed Report with UUID) logged.")
except Exception as e:
    print(f"Server error: {e}")

Server error: EndPointInternalError: The endpoint returned the HTTP status code 500. 

Response:
b'Entity pool initialization failure'


In [35]:
# Your final transformed dataset should also be documented appropriately using Croissant, SI, etc.

prepared_data_triples = [
    ':prepared_data rdf:type prov:Entity .',
    ':prepared_data rdf:type sc:Dataset .',
    ':prepared_data rdfs:label "Final Prepared Obesity Dataset" .',
    f':prepared_data rdfs:comment "Final dataset with {len(df)} rows. Includes BMI, age groups and encoded targets." .',

    # provenance: derived from raw data (2a), generated by prepare_data (3a)
    ':prepared_data prov:wasDerivedFrom :data .',
    ':prepared_data prov:wasGeneratedBy :prepare_data .',

    # 2. structure (croissant recordset)
    ':prepared_recordset rdf:type cr:RecordSet .',
    ':prepared_recordset sc:name "Prepared Data Records" .',
    ':prepared_data cr:recordSet :prepared_recordset .',

    # 3. describe new features
    # bmi with si unit
    ':prepared_recordset cr:field :field_bmi .',
    ':field_bmi rdf:type cr:Field .',
    ':field_bmi sc:name "BMI" .',
    ':field_bmi sc:description "Body Mass Index" .',
    ':field_bmi cr:dataType xsd:double .',
    ':field_bmi qudt:unit siu:KilogramPerSquareMetre .',

    # age group code (binned)
    ':prepared_recordset cr:field :field_age_group_code .',
    ':field_age_group_code rdf:type cr:Field .',
    ':field_age_group_code sc:name "Age_Group_Code" .',
    ':field_age_group_code sc:description "0=Youth, 1=YoungAdult, 2=Adult, 3=Senior" .',
    ':field_age_group_code cr:dataType xsd:integer .',

    # target encoded
    ':prepared_recordset cr:field :field_target_encoded .',
    ':field_target_encoded rdf:type cr:Field .',
    ':field_target_encoded sc:name "NObeyesdad" .',
    ':field_target_encoded sc:description "Target variable encoded (0-6)" .',
    ':field_target_encoded cr:dataType xsd:integer .',
]

try:
    engine.insert(prepared_data_triples, prefixes=prefixes)
    print("graph update: prepared data documented.")
except Exception as e:
    print(f"server error: {e}")

server error: EndPointInternalError: The endpoint returned the HTTP status code 500. 

Response:
b'Entity pool initialization failure'


## Modeling

In [36]:
## Each Activity that follows is part of the Modeling Phase

modeling_phase_executor = [
f':modeling_phase rdf:type prov:Activity .',
f':modeling rdfs:label "Modeling Phase" .', 
]
engine.insert(modeling_phase_executor, prefixes=prefixes)


EndPointInternalError: EndPointInternalError: The endpoint returned the HTTP status code 500. 

Response:
b'Entity pool initialization failure'

In [37]:
model_data_code_writer = student_a

#############################################
# documentation 4a
#############################################

# we use a fixed unique string for our group to avoid 403 errors
dma_ass_uuid_writer = "gr74-a-algo-selection-unique-id"

# rationale for choosing random forest
dma_comment = """
for the modeling phase, we selected the random forest classifier.
our dataset has 7 different obesity levels as target classes, and
about 77 percent of the data is synthetic generated by smote.
random forest is an ensemble method that is very robust against
overfitting on these synthetic patterns. it also handles the
outliers we found in the age and ncp attributes much better
than linear models. another advantage is that it provides
feature importance, which is great for our public health scenario
to see which habits have the most impact on obesity.
"""

identify_data_mining_algorithm_activity = [
    f':define_algorithm rdf:type prov:Activity .',
    f':define_algorithm sc:isPartOf :modeling_phase .',
    f':define_algorithm rdfs:label "task 4a algorithm selection" .',
    f':define_algorithm rdfs:comment """{dma_comment}""" .',

    # linking the activity to person a
    f':define_algorithm prov:qualifiedAssociation :{dma_ass_uuid_writer} .',
    f':{dma_ass_uuid_writer} prov:agent :{model_data_code_writer} .',
    f':{dma_ass_uuid_writer} rdf:type prov:Association .',
    f':{dma_ass_uuid_writer} prov:hadRole :{code_writer_role} .',

    # algorithm and implementation
    f':random_forest_algorithm rdf:type mls:Algorithm .',
    f':random_forest_algorithm rdfs:label "random forest" .',

    f':random_forrest_implementation rdf:type mls:Implementation .',
    f':random_forrest_implementation rdfs:label "scikit-learn randomforestclassifier" .',
    f':random_forrest_implementation mls:implements :random_forest_algorithm .',
    f':random_forrest_implementation prov:wasGeneratedBy :define_algorithm .',

    # defining evaluation measures for classification
    f':accuracy_measure rdf:type mls:EvaluationMeasure .',
    f':accuracy_measure rdfs:label "accuracy" .',
    f':accuracy_measure rdfs:comment "percentage of correct obesity class predictions" .',
    f':accuracy_measure prov:wasGeneratedBy :define_algorithm .',

    f':f1_macro_measure rdf:type mls:EvaluationMeasure .',
    f':f1_macro_measure rdfs:label "f1-score macro" .',
    f':f1_macro_measure rdfs:comment "macro-averaged f1 score for all 7 labels" .',
    f':f1_macro_measure prov:wasGeneratedBy :define_algorithm .'
]

# pushing the metadata to the graph
try:
    engine.insert(identify_data_mining_algorithm_activity, prefixes=prefixes)
    print("4a logged successfully: random forest selected.")
except:
    print("error: check if the node already exists or if there is a server issue.")

error: check if the node already exists or if there is a server issue.


In [38]:
# --- task 4b: hyper-parameter identification ---
# person a is responsible for identifying and justifying the parameters

# generate a fixed unique uuid for our group to avoid collisions
hp_ass_uuid_writer = "gr74-a-hp-selection-fixed"

# detailed rationale for tuning max_depth
# focuses on preventing overfitting on the 77% synthetic data
hp_comment = """
we identified several hyper-parameters for the random forest classifier,
including n_estimators, max_depth, and min_samples_split.
for our experiments, we select 'max_depth' as the primary parameter for tuning.
this choice is justified because it directly controls the complexity
of the individual trees. since the dataset contains 77 percent synthetic
records generated by smote, there is a high risk of the model learning
noise or artificial patterns. tuning max_depth allows us to find the
optimal balance between bias and variance and ensures better
generalization for real-world obesity screening.
"""

identify_hp_activity = [
    f':identify_hyperparameters rdf:type prov:Activity .',
    f':identify_hyperparameters sc:isPartOf :modeling_phase .',
    f':identify_hyperparameters rdfs:label "task 4b hyper-parameter identification" .',
    f':identify_hyperparameters rdfs:comment """{hp_comment}""" .',

    # link to person a with our fixed uuid
    f':identify_hyperparameters prov:qualifiedAssociation :{hp_ass_uuid_writer} .',
    f':{hp_ass_uuid_writer} prov:agent :{student_a} .',
    f':{hp_ass_uuid_writer} rdf:type prov:Association .',
    f':{hp_ass_uuid_writer} prov:hadRole :{code_writer_role} .',

    # define n_estimators as a relevant parameter
    f':hp_n_estimators rdf:type mls:HyperParameter .',
    f':hp_n_estimators rdfs:label "n_estimators" .',
    f':hp_n_estimators rdfs:comment "the number of trees in the forest." .',
    f':random_forrest_implementation mls:hasHyperParameter :hp_n_estimators .',
    f':hp_n_estimators prov:wasGeneratedBy :identify_hyperparameters .',

    # define max_depth as our tuning target
    f':hp_max_depth rdf:type mls:HyperParameter .',
    f':hp_max_depth rdfs:label "max_depth" .',
    f':hp_max_depth rdfs:comment "the maximum depth of the tree to control overfitting." .',
    f':random_forrest_implementation mls:hasHyperParameter :hp_max_depth .',
    f':hp_max_depth prov:wasGeneratedBy :identify_hyperparameters .'
]

# push the definitions to the graph
try:
    engine.insert(identify_hp_activity, prefixes=prefixes)
    print("4b logged successfully: hyperparameters identified and justified.")
except:
    print("error: check if the node already exists or server issues.")

error: check if the node already exists or server issues.


In [39]:
from sklearn.model_selection import train_test_split

# 4c: split logic
# we use 60% train, 20% validation, 20% test
def split_data(df: pd.DataFrame):
    x = df.drop(columns=['NObeyesdad'])
    y = df['NObeyesdad']

    # split test set first (20%)
    # stratify is key for our 7 obesity classes
    x_train_val, x_test, y_train_val, y_test = train_test_split(
        x, y, test_size=0.20, random_state=42, stratify=y
    )

    # split remaining 80% into train (60%) and val (20%)
    # 0.25 * 0.8 = 0.2
    x_train, x_val, y_train, y_val = train_test_split(
        x_train_val, y_train_val, test_size=0.25, random_state=42, stratify=y_train_val
    )

    return x_train, x_val, x_test, y_train, y_val, y_test

# execute split
x_train, x_val, x_test, y_train, y_val, y_test = split_data(df)

#############################################
# documentation 4c
#############################################

# fixed uuid for our group 74
split_ass_uuid_writer = "gr74-a-split-fixed-id"

# rationale for the split method
split_comment = """
we implemented a stratified 60/20/20 split to handle the 7 obesity classes.
stratification ensures that the distribution of obesity levels remains
consistent across train, validation, and test sets.
we used a fixed random seed (42) to ensure reproducibility as
required by the assignment.
"""

# set path for prepared data from phase 3
input_dataset = ":prepared_data"

define_split_activity = [
    f':define_data_split rdf:type prov:Activity .',
    f':define_data_split sc:isPartOf :modeling_phase .',
    f':define_data_split rdfs:label "task 4c data splitting" .',
    f':define_data_split rdfs:comment """{split_comment}""" .',
    f':define_data_split prov:qualifiedAssociation :{split_ass_uuid_writer} .',
    f':{split_ass_uuid_writer} prov:agent :{model_data_code_writer} .',
    f':{split_ass_uuid_writer} rdf:type prov:Association .',
    f':{split_ass_uuid_writer} prov:hadRole :{code_writer_role} .',
    f':define_data_split prov:used {input_dataset} .',

    # training set
    f':training_set rdf:type sc:Dataset .',
    f':training_set rdfs:label "training set" .',
    f':training_set prov:wasGeneratedBy :define_data_split .',
    f':training_set prov:wasDerivedFrom {input_dataset} .',
    f':training_set rdfs:comment "contains {len(x_train)} samples" .',

    # validation set
    f':validation_set rdf:type sc:Dataset .',
    f':validation_set rdfs:label "validation set" .',
    f':validation_set prov:wasGeneratedBy :define_data_split .',
    f':validation_set prov:wasDerivedFrom {input_dataset} .',
    f':validation_set rdfs:comment "contains {len(x_val)} samples" .',

    # test set
    f':test_set rdf:type sc:Dataset .',
    f':test_set rdfs:label "test set" .',
    f':test_set prov:wasGeneratedBy :define_data_split .',
    f':test_set prov:wasDerivedFrom {input_dataset} .',
    f':test_set rdfs:comment "contains {len(x_test)} samples" .',
]

try:
    engine.insert(define_split_activity, prefixes=prefixes)
    print("split documented. sizes: train", len(x_train), "val", len(x_val), "test", len(x_test))
except:
    print("error logging split - probably info already exists")

error logging split - probably info already exists


In [40]:
# --- task 4d, 4e & 4f: training, tuning and selection ---
# person a: running the tuning loop and selecting the best model

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 4d: define parameter space for tuning
# we must document all settings tested, not just defaults
depth_options = [3, 5, 10, 15, 20, None]
all_run_metadata = []

best_val_acc = 0
best_depth_val = None

# capture timing for the whole activity
start_time_tafm = now()

for d in depth_options:
    d_label = str(d) if d is not None else "none"

    # this creates the provenance for every failed and successful run
    run_id = f"run_rf_depth_{d_label}"
    model_id = f"model_rf_depth_{d_label}"
    hp_setting_id = f"hp_set_depth_{d_label}"
    eval_id = f"eval_acc_depth_{d_label}"

    # 1. training
    clf = RandomForestClassifier(max_depth=d, n_estimators=100, random_state=42)
    clf.fit(x_train, y_train)

    # 2. evaluation on validation set
    val_preds = clf.predict(x_val)
    acc = accuracy_score(y_val, val_preds)

    if acc > best_val_acc:
        best_val_acc = acc
        best_depth_val = d

    # 3. automate triple generation for this specific run
    all_run_metadata.extend([
        # parameter setting [cite: 110]
        f':{hp_setting_id} rdf:type mls:HyperParameterSetting .',
        f':{hp_setting_id} mls:specifiedBy :hp_max_depth .',
        f':{hp_setting_id} mls:hasValue "{d_label}" .',
        f':{hp_setting_id} prov:wasGeneratedBy :train_and_finetune_model .',

        # the run itself
        f':{run_id} rdf:type mls:Run .',
        f':{run_id} sc:isPartOf :train_and_finetune_model .',
        f':{run_id} mls:realizes :random_forest_algorithm .',
        f':{run_id} mls:hasInput :training_set .',
        f':{run_id} mls:hasInput :{hp_setting_id} .',
        f':{run_id} mls:hasOutput :{model_id} .',
        f':{run_id} mls:hasOutput :{eval_id} .',

        # the resulting model [cite: 113]
        f':{model_id} rdf:type mls:Model .',
        f':{model_id} prov:wasGeneratedBy :{run_id} .',
        f':{model_id} mlso:trainedOn :training_set .',

        # the evaluation result [cite: 111]
        f':{eval_id} rdf:type mls:ModelEvaluation .',
        f':{eval_id} prov:wasGeneratedBy :{run_id} .',
        f':{eval_id} mls:hasValue "{acc}"^^xsd:double .',
        f':{eval_id} mls:specifiedBy :accuracy_measure .',
        f':{eval_id} prov:used :validation_set .'
    ])

end_time_tafm = now()

#############################################
# final documentation list
#############################################

# fixed id for our group
tafm_ass_uuid_writer = "gr74-a-tuning-session-fixed"

# 4f: document the decision for the best model
tafm_comment = f"""
we tested max_depth levels from 3 to none.
the best performance on the validation set was {best_val_acc:.4f}
achieved with max_depth={best_depth_val}.
this model is selected for final evaluation as it balances
complexity and accuracy effectively.
"""

# this list contains the main activity info
train_model_activity_main = [
    f':train_and_finetune_model rdf:type prov:Activity .',
    f':train_and_finetune_model sc:isPartOf :modeling_phase .',
    f':train_and_finetune_model rdfs:label "task 4d & 4e training and tuning" .',
    f':train_and_finetune_model rdfs:comment """{tafm_comment}""" .',
    f':train_and_finetune_model prov:startedAtTime "{start_time_tafm}"^^xsd:dateTime .',
    f':train_and_finetune_model prov:endedAtTime "{end_time_tafm}"^^xsd:dateTime .',

    f':train_and_finetune_model prov:qualifiedAssociation :{tafm_ass_uuid_writer} .',
    f':{tafm_ass_uuid_writer} prov:agent :{student_a} .',
    f':{tafm_ass_uuid_writer} rdf:type prov:Association .',
    f':{tafm_ass_uuid_writer} prov:hadRole :{code_writer_role} .',
]

# join the main activity and all automated runs into one list
# this is why the 'variable' looked shorter before, but the content is huge!
full_modeling_triples = train_model_activity_main + all_run_metadata

try:
    engine.insert(full_modeling_triples, prefixes=prefixes)
    print(f"logged all {len(depth_options)} runs. best depth: {best_depth_val}")
except:
    print("graph error - check for duplicate uris if re-running")

graph error - check for duplicate uris if re-running


In [41]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

# --- task 4g: final model retraining ---

# combine sets for final training
x_final_train = pd.concat([x_train, x_val])
y_final_train = pd.concat([y_train, y_val])

# use the best depth found in 4d
final_clf = RandomForestClassifier(
    max_depth=best_depth_val,
    n_estimators=100,
    random_state=42
)

start_time_final = now()
final_clf.fit(x_final_train, y_final_train)
end_time_final = now()

#############################################
# documentation 4g
#############################################

# using the provided fixed uuid
retrain_ass_uuid_writer = "96815ee0-524c-437b-b5fa-2e15b945c993"

# simple rationale for retraining
final_model_comment = f"""
retrained the final random forest model on the complete
training and validation data using max_depth={best_depth_val}.
"""

retrain_documentation = [
    f':retrain_final_model rdf:type prov:Activity .',
    f':retrain_final_model sc:isPartOf :modeling_phase .',
    f':retrain_final_model rdfs:label "task 4g: final retraining" .',
    f':retrain_final_model rdfs:comment """{final_model_comment}""" .',
    f':retrain_final_model prov:startedAtTime "{start_time_final}"^^xsd:dateTime .',
    f':retrain_final_model prov:endedAtTime "{end_time_final}"^^xsd:dateTime .',

    # link to person a
    f':retrain_final_model prov:qualifiedAssociation :{retrain_ass_uuid_writer} .',
    f':{retrain_ass_uuid_writer} prov:agent :{student_a} .',
    f':{retrain_ass_uuid_writer} rdf:type prov:Association .',
    f':{retrain_ass_uuid_writer} prov:hadRole :{code_writer_role} .',

    # inputs and output
    f':retrain_final_model prov:used :training_set .',
    f':retrain_final_model prov:used :validation_set .',
    f':final_model_entity rdf:type mls:Model .',
    f':final_model_entity prov:wasGeneratedBy :retrain_final_model .',
    f':final_model_entity mlso:trainedOn :training_set .'
]

try:
    engine.insert(retrain_documentation, prefixes=prefixes)
    print("4g logged: final model created and stored.")
except:
    print("graph error - check for duplicate nodes.")

graph error - check for duplicate nodes.


## Evaluation

In [42]:
## Each Activity that follows is part of the Evaluation Phase

evaluation_phase_executor = [
f':evaluation_phase rdf:type prov:Activity .',
f':evaluation_phase rdfs:label "Evaluation Phase" .', 
]
engine.insert(evaluation_phase_executor, prefixes=prefixes)

EndPointInternalError: EndPointInternalError: The endpoint returned the HTTP status code 500. 

Response:
b'Entity pool initialization failure'

In [43]:
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# --- task 5: evaluation ---
# person b is responsible for this phase

def evaluate_on_test_data(model, x_test, y_test):
    # predict on test data
    y_pred = model.predict(x_test)
    acc = accuracy_score(y_test, y_pred)

    # 5e: bias evaluation (checking gender bias)
    test_df = x_test.copy()
    test_df['target'] = y_test
    test_df['pred'] = y_pred

    # gender 0 = female, 1 = male
    acc_female = accuracy_score(test_df[test_df['Gender'] <= 0]['target'],
                                test_df[test_df['Gender'] <= 0]['pred'])
    acc_male = accuracy_score(test_df[test_df['Gender'] > 0]['target'],
                              test_df[test_df['Gender'] > 0]['pred'])

    bias_report = f"accuracy female: {acc_female:.4f}, accuracy male: {acc_male:.4f}"
    return acc, bias_report

eval_code_writer = student_b
start_time_eval = now()
# using final_clf from task 4g
test_performance, gender_bias_results = evaluate_on_test_data(final_clf, x_test, y_test)
end_time_eval = now()

#############################################
# documentation
#############################################

# changed to a fixed unique id for group 74
eval_ass_uuid = "gr74-b-eval-final-fixed"
final_model = ":final_model_entity"
test_set = ":test_set"

eval_comment = f"""
the final random forest model achieved a test accuracy of {test_performance:.4f}.
this meets our data mining success criteria.
we compared it against kaggle benchmarks (95-99%) and a random baseline.
"""

evaluate_activity = [
    f':evaluation_phase rdf:type prov:Activity .',
    f':evaluation_phase rdfs:label "evaluation phase" .',

    f':evaluate_final_model rdf:type prov:Activity .',
    f':evaluate_final_model sc:isPartOf :evaluation_phase .',
    f':evaluate_final_model rdfs:label "final model evaluation on test set" .',
    f':evaluate_final_model rdfs:comment """{eval_comment}""" .',
    f':evaluate_final_model prov:startedAtTime "{start_time_eval}"^^xsd:dateTime .',
    f':evaluate_final_model prov:endedAtTime "{end_time_eval}"^^xsd:dateTime .',

    # link to person b with our new fixed id
    f':evaluate_final_model prov:qualifiedAssociation :{eval_ass_uuid} .',
    f':{eval_ass_uuid} prov:agent :{eval_code_writer} .',
    f':{eval_ass_uuid} rdf:type prov:Association .',
    f':{eval_ass_uuid} prov:hadRole :{code_writer_role} .',

    # inputs
    f':evaluate_final_model prov:used {final_model} .',
    f':evaluate_final_model prov:used {test_set} .',

    # metrics
    f':test_performance_result rdf:type mls:ModelEvaluation .',
    f':test_performance_result mls:hasValue "{test_performance}"^^xsd:double .',
    f':test_performance_result mls:specifiedBy :accuracy_measure .',
    f':test_performance_result prov:wasGeneratedBy :evaluate_final_model .',

    # 5e: bias analysis
    f':bias_evaluation_result rdf:type mls:ModelEvaluation .',
    f':bias_evaluation_result prov:wasGeneratedBy :evaluate_final_model .',
    f':bias_evaluation_result rdfs:label "bias analysis (gender)" .',
    f':bias_evaluation_result rdfs:comment "{gender_bias_results}" .',
]

try:
    engine.insert(evaluate_activity, prefixes=prefixes)
    print(f"evaluation logged with fixed id. accuracy: {test_performance:.4f}")
except:
    print("error - check if uris already exist")

error - check if uris already exist


In [44]:
#############################################
# TASK 5b: BASELINE AND SOTA PERFORMANCE
#############################################

from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

print("\n=== Task 5b: Baseline and SOTA Evaluation ===\n")

# Calculate baselines
random_clf = DummyClassifier(strategy='uniform', random_state=42)
random_clf.fit(x_train, y_train)
random_acc = accuracy_score(y_test, random_clf.predict(x_test))

majority_clf = DummyClassifier(strategy='most_frequent', random_state=42)
majority_clf.fit(x_train, y_train)
majority_acc = accuracy_score(y_test, majority_clf.predict(x_test))

print(f"Random Baseline:   {random_acc:.4f}")
print(f"Majority Baseline: {majority_acc:.4f}")
print(f"Our Model:         {test_performance:.4f}")
print(f"Improvement:       {(test_performance - random_acc)*100:.1f} percentage points")
print()

# SOTA documentation
sota_comment = """
State-of-the-Art Research:
- Palechor 2019 (original paper): 97.14% accuracy with MLP, 95.71% with Decision Trees
- Kaggle competitions: 95-99% accuracy range
- Our model targets >=90% as per success criteria
Source: Palechor & De la Hoz Manotas (2019), Data in Brief, Vol 25
"""
print("SOTA Benchmark: Palechor 2019 achieved 97.14% (MLP)")
print()

# Provenance
baseline_ass_uuid = "gr74-b-baseline-eval-fixed"
baseline_comment = f"Baseline evaluation: Random={random_acc:.4f}, Majority={majority_acc:.4f}, Our model={test_performance:.4f}. Model exceeds random baseline by {(test_performance - random_acc)*100:.1f} percentage points. SOTA from literature: 97.14% (Palechor 2019)."

baseline_activity = [
    f':evaluate_baselines rdf:type prov:Activity .',
    f':evaluate_baselines sc:isPartOf :evaluation_phase .',
    f':evaluate_baselines rdfs:label "Task 5b: Baseline and SOTA Evaluation" .',
    f':evaluate_baselines rdfs:comment "{baseline_comment}" .',
    f':evaluate_baselines prov:qualifiedAssociation :{baseline_ass_uuid} .',
    f':{baseline_ass_uuid} prov:agent :{student_b} .',
    f':{baseline_ass_uuid} rdf:type prov:Association .',
    f':{baseline_ass_uuid} prov:hadRole :{code_writer_role} .',
    f':random_baseline_result rdf:type mls:ModelEvaluation .',
    f':random_baseline_result mls:hasValue "{random_acc}"^^xsd:double .',
    f':random_baseline_result rdfs:label "Random Baseline" .',
    f':random_baseline_result prov:wasGeneratedBy :evaluate_baselines .',
    f':majority_baseline_result rdf:type mls:ModelEvaluation .',
    f':majority_baseline_result mls:hasValue "{majority_acc}"^^xsd:double .',
    f':majority_baseline_result rdfs:label "Majority Baseline" .',
    f':majority_baseline_result prov:wasGeneratedBy :evaluate_baselines .',
]

try:
    engine.insert(baseline_activity, prefixes=prefixes)
    print("Task 5b logged to GraphDB")
except Exception as e:
    print(f"Error: {e}")


=== Task 5b: Baseline and SOTA Evaluation ===

Random Baseline:   0.1722
Majority Baseline: 0.1675
Our Model:         0.9856
Improvement:       81.3 percentage points

SOTA Benchmark: Palechor 2019 achieved 97.14% (MLP)

Error: EndPointInternalError: The endpoint returned the HTTP status code 500. 

Response:
b'Entity pool initialization failure'


In [45]:
#############################################
# TASK 5c: DETAILED PERFORMANCE COMPARISON
#############################################

from sklearn.metrics import confusion_matrix, classification_report, f1_score, precision_score, recall_score
import matplotlib.pyplot as plt
import os

print("\n=== Task 5c: Detailed Performance Comparison ===\n")

y_pred = final_clf.predict(x_test)
class_names = ['Insufficient_Weight', 'Normal_Weight', 'Overweight_Level_I',
               'Overweight_Level_II', 'Obesity_Type_I', 'Obesity_Type_II', 'Obesity_Type_III']

# 1. Confusion Matrix (simplified plot)
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
plt.imshow(cm, cmap='Blues')
plt.colorbar()
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
for i in range(len(cm)):
    for j in range(len(cm)):
        plt.text(j, i, str(cm[i,j]), ha='center', va='center')
plt.tight_layout()
os.makedirs('data/figures', exist_ok=True)
plt.savefig('data/figures/confusion_matrix.png', dpi=150)
print("Confusion matrix saved")
plt.close()

# 2. Per-class metrics (text only, no plots)
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=class_names))

# 3. Calculate aggregate metrics
macro_f1 = f1_score(y_test, y_pred, average='macro')
macro_precision = precision_score(y_test, y_pred, average='macro')
macro_recall = recall_score(y_test, y_pred, average='macro')

print(f"\nMacro F1:        {macro_f1:.4f}")
print(f"Macro Precision: {macro_precision:.4f}")
print(f"Macro Recall:    {macro_recall:.4f}")
print()

# Provenance
detailed_comp_uuid = "gr74-b-detailed-comparison-fixed"
detailed_comp_comment = f"Confusion matrix analysis completed. Macro metrics: F1={macro_f1:.4f}, Precision={macro_precision:.4f}, Recall={macro_recall:.4f}. Model shows balanced performance across all 7 obesity classes."

detailed_comp_activity = [
    f':detailed_performance_comparison rdf:type prov:Activity .',
    f':detailed_performance_comparison sc:isPartOf :evaluation_phase .',
    f':detailed_performance_comparison rdfs:label "Task 5c: Detailed Performance Comparison" .',
    f':detailed_performance_comparison rdfs:comment "{detailed_comp_comment}" .',
    f':detailed_performance_comparison prov:qualifiedAssociation :{detailed_comp_uuid} .',
    f':{detailed_comp_uuid} prov:agent :{student_b} .',
    f':{detailed_comp_uuid} rdf:type prov:Association .',
    f':{detailed_comp_uuid} prov:hadRole :{code_writer_role} .',
    f':confusion_matrix_analysis rdf:type prov:Entity .',
    f':confusion_matrix_analysis prov:wasGeneratedBy :detailed_performance_comparison .',
    f':confusion_matrix_analysis rdfs:label "Confusion Matrix" .',
    f':macro_f1_result rdf:type mls:ModelEvaluation .',
    f':macro_f1_result mls:hasValue "{macro_f1}"^^xsd:double .',
    f':macro_f1_result rdfs:label "Macro F1-score" .',
    f':macro_f1_result prov:wasGeneratedBy :detailed_performance_comparison .',
]

try:
    engine.insert(detailed_comp_activity, prefixes=prefixes)
    print("Task 5c logged to GraphDB")
except Exception as e:
    print(f"Error: {e}")


=== Task 5c: Detailed Performance Comparison ===

Confusion matrix saved

Classification Report:
                     precision    recall  f1-score   support

Insufficient_Weight       1.00      1.00      1.00        53
      Normal_Weight       0.95      1.00      0.97        57
 Overweight_Level_I       1.00      1.00      1.00        70
Overweight_Level_II       0.98      1.00      0.99        60
     Obesity_Type_I       1.00      0.98      0.99        65
    Obesity_Type_II       1.00      0.95      0.97        55
   Obesity_Type_III       0.97      0.97      0.97        58

           accuracy                           0.99       418
          macro avg       0.99      0.99      0.99       418
       weighted avg       0.99      0.99      0.99       418


Macro F1:        0.9851
Macro Precision: 0.9856
Macro Recall:    0.9851

Error: EndPointInternalError: The endpoint returned the HTTP status code 500. 

Response:
b'Entity pool initialization failure'


In [46]:
#############################################
# TASK 5d: COMPARE WITH BUSINESS SUCCESS CRITERIA (SIMPLIFIED)
#############################################

from sklearn.metrics import accuracy_score

print("\n=== Task 5d: Success Criteria Comparison ===\n")

# Recalculate gender bias
test_df_bias = x_test.copy()
test_df_bias['target'] = y_test.values
test_df_bias['pred'] = final_clf.predict(x_test)
acc_female = accuracy_score(test_df_bias[test_df_bias['Gender'] <= 0]['target'],
                            test_df_bias[test_df_bias['Gender'] <= 0]['pred'])
acc_male = accuracy_score(test_df_bias[test_df_bias['Gender'] > 0]['target'],
                          test_df_bias[test_df_bias['Gender'] > 0]['pred'])

# Check criteria
print("DATA MINING SUCCESS CRITERIA:")
print(f"1. Accuracy >=90%: {test_performance:.4f} - {'MET' if test_performance >= 0.90 else 'NOT MET'}")
print(f"2. Macro F1 >=0.85: {macro_f1:.4f} - {'MET' if macro_f1 >= 0.85 else 'NOT MET'}")
print(f"3. Baseline improvement >60pp: {(test_performance - random_acc)*100:.1f}pp - {'MET' if (test_performance - random_acc) > 0.60 else 'NOT MET'}")
print()

print("BUSINESS OBJECTIVES:")
print(f"1. Early Risk ID: {'ACHIEVED' if test_performance >= 0.90 else 'PARTIAL'}")
print(f"2. Resource Allocation: {'ACHIEVED' if macro_f1 >= 0.85 else 'PARTIAL'}")
print(f"3. Interpretability: ACHIEVED (Random Forest)")
print(f"4. Gender Fairness: Female={acc_female:.4f}, Male={acc_male:.4f}, Gap={abs(acc_female-acc_male):.4f}")
print()

# Deployment recommendation
if test_performance >= 0.90 and macro_f1 >= 0.85:
    recommendation = "Hybrid deployment with human oversight"
else:
    recommendation = "Limited deployment for screening only"
print(f"RECOMMENDATION: {recommendation}")
print()

# Provenance
success_criteria_uuid = "gr74-b-success-criteria-comparison-fixed"
success_criteria_comment = f"Success criteria comparison: Accuracy={test_performance:.4f} (target >=0.90), Macro F1={macro_f1:.4f} (target >=0.85). Gender bias: Female={acc_female:.4f}, Male={acc_male:.4f}. Deployment: {recommendation}."

success_criteria_activity = [
    f':compare_success_criteria rdf:type prov:Activity .',
    f':compare_success_criteria sc:isPartOf :evaluation_phase .',
    f':compare_success_criteria rdfs:label "Task 5d: Success Criteria Comparison" .',
    f':compare_success_criteria rdfs:comment "{success_criteria_comment}" .',
    f':compare_success_criteria prov:qualifiedAssociation :{success_criteria_uuid} .',
    f':{success_criteria_uuid} prov:agent :{student_b} .',
    f':{success_criteria_uuid} rdf:type prov:Association .',
    f':{success_criteria_uuid} prov:hadRole :{code_writer_role} .',
    f':compare_success_criteria prov:used :bu_business_success_criteria .',
    f':compare_success_criteria prov:used :bu_data_mining_success_criteria .',
    f':success_criteria_assessment rdf:type prov:Entity .',
    f':success_criteria_assessment prov:wasGeneratedBy :compare_success_criteria .',
    f':success_criteria_assessment rdfs:label "Success Criteria Assessment" .',
]

try:
    engine.insert(success_criteria_activity, prefixes=prefixes)
    print("Task 5d logged to GraphDB")
except Exception as e:
    print(f"Error: {e}")


=== Task 5d: Success Criteria Comparison ===

DATA MINING SUCCESS CRITERIA:
1. Accuracy >=90%: 0.9856 - MET
2. Macro F1 >=0.85: 0.9851 - MET
3. Baseline improvement >60pp: 81.3pp - MET

BUSINESS OBJECTIVES:
1. Early Risk ID: ACHIEVED
2. Resource Allocation: ACHIEVED
3. Interpretability: ACHIEVED (Random Forest)
4. Gender Fairness: Female=0.9798, Male=0.9909, Gap=0.0111

RECOMMENDATION: Hybrid deployment with human oversight

Error: EndPointInternalError: The endpoint returned the HTTP status code 500. 

Response:
b'Entity pool initialization failure'


## Deployment

In [47]:
## Each Activity that follows is part of the Deployment Phase

deployment_phase_executor = [
f':deployment_phase rdf:type prov:Activity .',
f':deployment_phase rdfs:label "Deployment Phase" .', 
]
engine.insert(deployment_phase_executor, prefixes=prefixes)

EndPointInternalError: EndPointInternalError: The endpoint returned the HTTP status code 500. 

Response:
b'Entity pool initialization failure'

In [48]:
#############################################
# documentation phase 6: deployment
#############################################

# 6a: reflection on business objectives and success criteria
# compare performance to the goals from phase 1
comparison_and_recommendations_comment = """
our final random forest model achieved a test accuracy of over 90 percent,
which successfully fulfills the business success criteria defined during
the first phase of crisp-dm. since the performance is consistently high
across all seven obesity levels, the tool provides a solid foundation for
public health agencies to identify at-risk populations early. we recommend
a hybrid deployment strategy where the model acts as a preliminary
screening tool for clinics in mexico, peru, and colombia. however,
automated results must always be verified by healthcare professionals to
avoid misdiagnosis, especially for minority classes or unusual lifestyle patterns.
"""

# 6b: ethical aspects and risks identified for deployment
# mention the smote data and geographic limitations
ethical_aspects_comment = """
the most significant ethical concern is the use of 77 percent synthetic
data generated via smote. while this helps with class balance, it might
introduce artificial patterns that do not perfectly reflect the biological
diversity of real patients. furthermore, the model is geographically
limited to three latin american countries, which could lead to bias if
applied to other regions with different dietary cultures. we must also
ensure that the classification does not lead to patient stigmatization
or discrimination in insurance contexts, requiring strict data
privacy protocols and human oversight.
"""

# 6c: monitoring plan during deployment
# define triggers for intervention
monitoring_plan_comment = """
to maintain model reliability, we propose a two-tier monitoring plan.
first, we must monitor for data drift, specifically tracking if the
distribution of eating habits or transportation modes in new patients
shifts significantly from our training set. second, we define a
performance trigger: we will perform regular audits against manual medical
labels. if the classification accuracy for any specific subgroup or
the overall population drops below 85 percent, it will trigger an
immediate review and potential retraining of the model.
"""

# 6d: reflection on reproducibility
# how easy is it to replicate our results?
reproducibility_reflection_comment = """
reproducibility of our experiment is high because we used fixed random
seeds (42) for all data splits and training runs. every processing step,
from initial data cleaning and bmi calculation to the final hyperparameter
tuning of the random forest, is documented within this provenance
knowledge graph. a remaining risk for reproducibility lies in the
dependency on specific library versions for scaling and preprocessing,
which must be documented clearly in the final report to ensure
consistent results across different environments.
"""

# fixed unique id for our group 74
dep_ass_uuid_executor = "gr74-ab-deployment-final-fixed"

deployment_executor = [
    f':deployment_phase rdf:type prov:Activity .',
    f':deployment_phase rdfs:label "deployment phase" .',

    f':plan_deployment rdf:type prov:Activity .',
    f':plan_deployment sc:isPartOf :deployment_phase .',
    f':plan_deployment rdfs:label "plan deployment" .',

    f':plan_deployment prov:qualifiedAssociation :{dep_ass_uuid_executor} .',
    f':{dep_ass_uuid_executor} prov:agent :{executed_by} .',
    f':{dep_ass_uuid_executor} rdf:type prov:Association .',
    f':{dep_ass_uuid_executor} prov:hadRole :{code_executor_role} .',
]

try:
    engine.insert(deployment_executor, prefixes=prefixes)
    print("deployment activity logged")
except:
    print("activity already exists - check uris")


deployment_data_executor = [
    # 6a
    f':dep_recommendations rdf:type prov:Entity .',
    f':dep_recommendations prov:wasGeneratedBy :plan_deployment .',
    f':dep_recommendations rdfs:label "6a business objectives reflection" .',
    f':dep_recommendations rdfs:comment """{comparison_and_recommendations_comment}""" .',
    # 6b
    f':dep_ethical_risks rdf:type prov:Entity .',
    f':dep_ethical_risks prov:wasGeneratedBy :plan_deployment .',
    f':dep_ethical_risks rdfs:label "6b ethical aspects and risks" .',
    f':dep_ethical_risks rdfs:comment """{ethical_aspects_comment}""" .',
    # 6c
    f':dep_monitoring_plan rdf:type prov:Entity .',
    f':dep_monitoring_plan prov:wasGeneratedBy :plan_deployment .',
    f':dep_monitoring_plan rdfs:label "6c monitoring plan" .',
    f':dep_monitoring_plan rdfs:comment """{monitoring_plan_comment}""" .',
    # 6d
    f':dep_reproducibility_reflection rdf:type prov:Entity .',
    f':dep_reproducibility_reflection prov:wasGeneratedBy :plan_deployment .',
    f':dep_reproducibility_reflection rdfs:label "6d reproducibility reflection" .',
    f':dep_reproducibility_reflection rdfs:comment """{reproducibility_reflection_comment}""" .',
]

try:
    engine.insert(deployment_data_executor, prefixes=prefixes)
    print("deployment data logged successfully")
except:
    print("entities already exist in graph")

activity already exists - check uris
entities already exist in graph


# Generate Latex Report

The following cells give you an example of how to automatically create a Latex Report from your provenance documentation.

Feel free to use the example provided. If you use it, you should adapt and extend it with relevant sections/tables/plots/... 

In [49]:
base_iri = f"https://starvers.ec.tuwien.ac.at/BI2025/{group_id}/"

In [50]:
# This cell includes cleaning functions

from datetime import datetime

def latex_escape(text: str | None) -> str:
    if text is None: return ""
    text = str(text)
    text = text.replace("\\", r"\textbackslash{}")
    pairs = [
        ("&", r"\&"), ("%", r"\%"), ("$", r"\$"), ("#", r"\#"), 
        ("_", r"\_"), ("{", r"\{"), ("}", r"\}"), 
        ("~", r"\textasciitilde{}"), ("^", r"\textasciicircum{}")
    ]
    for k, v in pairs:
        text = text.replace(k, v)
    return text

def clean_rdf(x) -> str:
    if hasattr(x, "toPython"): return str(x.toPython())
    if x is None: return ""
    s = str(x).strip()
    s = s.strip('"').strip("'")
    s = s.strip()
    if "^^" in s:
        s = s.split("^^")[0].strip('"')
        
    return s

def fmt_iso(ts: str) -> str:
    if not ts: return ""
    try:
        clean_ts = ts.split("^^")[0].strip('"')
        clean_ts = clean_ts.replace("Z", "+00:00") if clean_ts.endswith("Z") else clean_ts
        return datetime.fromisoformat(clean_ts).strftime("%Y-%m-%d %H:%M:%S")
    except:
        return latex_escape(str(ts))

In [52]:
# ++++++++++++++++++++++++++++++++++ FINAL QUERIES (Phases 1-6) +++++++++++++++++++++++++++++++++++++++++++++

# 1. Authors & Business Understanding (Sections 1.1 - 1.4)
author_query = f"""{prefix_header} PREFIX iao: <http://purl.obolibrary.org/obo/>
SELECT DISTINCT ?uri ?given ?family ?matr WHERE {{
  VALUES ?uri {{ :{student_a} :{student_b} }}
  ?uri a foaf:Person ; foaf:givenName ?given ; foaf:familyName ?family ; iao:IAO_0000219 ?matr .
}}"""
res_authors = engine.query(author_query)
author_block_latex = ""
if not res_authors.empty:
    for _, row in res_authors.iterrows():
        given, family, matr = [latex_escape(clean_rdf(row[k])) for k in ['given', 'family', 'matr']]
        resp = "Student A" if student_a in str(row['uri']) else "Student B"
        author_block_latex += rf"\author{{{given} {family}}} \authornote{{{resp}, Matr.Nr.: {matr}}} \affiliation{{\institution{{TU Wien}} \country{{Austria}}}}"

bu_query = f"{prefix_header} SELECT ?ds ?bo ?bsc ?dmg WHERE {{ OPTIONAL {{ :bu_data_source_and_scenario rdfs:comment ?ds . }} OPTIONAL {{ :bu_business_objectives rdfs:comment ?bo . }} OPTIONAL {{ :bu_business_success_criteria rdfs:comment ?bsc . }} OPTIONAL {{ :bu_data_mining_goals rdfs:comment ?dmg . }} }} LIMIT 1"
res_bu = engine.query(bu_query)
row_bu = res_bu.iloc[0] if not res_bu.empty else {}
bu_data_source, bu_objectives, bu_success_crit, bu_mining_goals = [latex_escape(clean_rdf(row_bu.get(k, ""))) for k in ["ds", "bo", "bsc", "dmg"]]

# 2. Data Understanding (Section 2.1 - 2.7)
du_desc_query = f"{prefix_header} SELECT ?desc WHERE {{ :data sc:description ?desc . }} LIMIT 1"
du_description = latex_escape(clean_rdf(engine.query(du_desc_query).iloc[0].get("desc", ""))) if not engine.query(du_desc_query).empty else ""

# 2a Table
du_fields_query = f"{prefix_header} SELECT ?name (SAMPLE(?dtypeRaw) as ?dtype) (SAMPLE(?descRaw) as ?desc) WHERE {{ :data cr:recordSet ?rs . ?rs cr:field ?field . ?field sc:name ?name ; sc:description ?descRaw ; cr:dataType ?dtypeRaw . }} GROUP BY ?name ORDER BY ?name"
res_du = engine.query(du_fields_query)
du_table_rows = "\n    ".join([f"{latex_escape(clean_rdf(f['name']))} & {latex_escape(clean_rdf(f['dtype']).split('#')[-1])} & {latex_escape(clean_rdf(f['desc']))} \\\\" for _, f in res_du.iterrows()]) if not res_du.empty else ""

# 2b-2g Summaries
def get_comm(uri): return latex_escape(clean_rdf(engine.query(f"{prefix_header} SELECT ?c WHERE {{ {uri} rdfs:comment ?c . }} LIMIT 1").iloc[0].get("c", ""))) if not engine.query(f"{prefix_header} SELECT ?c WHERE {{ {uri} rdfs:comment ?c . }} LIMIT 1").empty else ""
du_statistics_summary = get_comm(":analyze_statistics")
du_quality_summary = get_comm(":assess_data_quality")
du_ethics_summary = get_comm(":assess_ethical_sensitivity")
du_bias_summary = get_comm(":analyze_bias_risks")
du_prep_plan = get_comm(":plan_data_preparation")

# 2d - Visual Exploration Path
res_du_viz = engine.query(f"{prefix_header} SELECT ?comment ?url WHERE {{ :explore_visually rdfs:comment ?comment . OPTIONAL {{ :visualization_report sc:contentUrl ?url . }} }} LIMIT 1")
row_du_viz = res_du_viz.iloc[0] if not res_du_viz.empty else {}
du_viz_summary = latex_escape(clean_rdf(row_du_viz.get("comment", "")))
du_viz_path = clean_rdf(row_du_viz.get("url", "")).replace("file://", "")

# 3. Data Preparation (Section 3.1 - 3.4)
dp_res = engine.query(f"{prefix_header} SELECT ?p ?r ?d ?e WHERE {{ OPTIONAL {{ :prepare_data rdfs:comment ?p . }} OPTIONAL {{ :data_prep_not_applied rdfs:comment ?r . }} OPTIONAL {{ :data_prep_derived_attrs rdfs:comment ?d . }} OPTIONAL {{ :data_prep_external_data rdfs:comment ?e . }} }} LIMIT 1")
row_dp = dp_res.iloc[0] if not dp_res.empty else {}
dp_summary, dp_rejected, dp_derived, dp_external = [latex_escape(clean_rdf(row_dp.get(k, ""))) for k in ["p", "r", "d", "e"]]

# 4. Modeling (Algorithm, Split, Tuning-Plot)
mod_res = engine.query(f"{prefix_header} SELECT ?a ?h ?s ?r WHERE {{ OPTIONAL {{ :define_algorithm rdfs:comment ?a . }} OPTIONAL {{ :identify_hyperparameters rdfs:comment ?h . }} OPTIONAL {{ :define_data_split rdfs:comment ?s . }} OPTIONAL {{ :retrain_final_model rdfs:comment ?r . }} }} LIMIT 1")
row_mod = mod_res.iloc[0] if not mod_res.empty else {}
mod_algo_text, mod_hp_text, mod_split_text, mod_retrain_text = [latex_escape(clean_rdf(row_mod.get(k, ""))) for k in ["a", "h", "s", "r"]]

# --- NEW: Get Tuning Plot Path (Task 4d) ---
res_tuning_plot = engine.query(f"{prefix_header} SELECT ?url WHERE {{ :tuning_plot_entity sc:contentUrl ?url . }} LIMIT 1")
final_plot_path_clean = clean_rdf(res_tuning_plot.iloc[0].get("url", "")).replace("file://", "") if not res_tuning_plot.empty else ""

# 5. Evaluation (Performance & Bias)
eval_res = engine.query(f"{prefix_header} SELECT ?e ?p ?b WHERE {{ OPTIONAL {{ :evaluate_final_model rdfs:comment ?e . }} OPTIONAL {{ :test_performance_result mls:hasValue ?p . }} OPTIONAL {{ :bias_evaluation_result rdfs:comment ?b . }} }} LIMIT 1")
row_eval = eval_res.iloc[0] if not eval_res.empty else {}
eval_main_text = latex_escape(clean_rdf(row_eval.get("e", "")))
eval_perf_val = clean_rdf(row_eval.get("p", ""))
eval_bias_text = latex_escape(clean_rdf(row_eval.get("b", "")))

# 5b. Baseline and SOTA Evaluation
baseline_res = engine.query(f"{prefix_header} SELECT ?comm ?rand ?maj WHERE {{ OPTIONAL {{ :evaluate_baselines rdfs:comment ?comm . }} OPTIONAL {{ :random_baseline_result mls:hasValue ?rand . }} OPTIONAL {{ :majority_baseline_result mls:hasValue ?maj . }} }} LIMIT 1")
row_baseline = baseline_res.iloc[0] if not baseline_res.empty else {}
eval_baseline_text = latex_escape(clean_rdf(row_baseline.get("comm", "")))
eval_random_val = clean_rdf(row_baseline.get("rand", ""))
eval_majority_val = clean_rdf(row_baseline.get("maj", ""))

# 5c. Detailed Performance Comparison (Confusion Matrix)
detailed_res = engine.query(f"{prefix_header} SELECT ?comm ?f1 WHERE {{ OPTIONAL {{ :detailed_performance_comparison rdfs:comment ?comm . }} OPTIONAL {{ :macro_f1_result mls:hasValue ?f1 . }} }} LIMIT 1")
row_detailed = detailed_res.iloc[0] if not detailed_res.empty else {}
eval_detailed_text = latex_escape(clean_rdf(row_detailed.get("comm", "")))
eval_macro_f1_val = clean_rdf(row_detailed.get("f1", ""))

# 5d. Success Criteria Comparison
success_res = engine.query(f"{prefix_header} SELECT ?comm WHERE {{ OPTIONAL {{ :compare_success_criteria rdfs:comment ?comm . }} }} LIMIT 1")
row_success = success_res.iloc[0] if not success_res.empty else {}
eval_success_text = latex_escape(clean_rdf(row_success.get("comm", "")))


# 6. Deployment
dep_res = engine.query(f"{prefix_header} SELECT ?r ?e ?m ?p WHERE {{ OPTIONAL {{ :dep_recommendations rdfs:comment ?r . }} OPTIONAL {{ :dep_ethical_risks rdfs:comment ?e . }} OPTIONAL {{ :dep_monitoring_plan rdfs:comment ?m . }} OPTIONAL {{ :dep_reproducibility_reflection rdfs:comment ?p . }} }} LIMIT 1")
row_dep = dep_res.iloc[0] if not dep_res.empty else {}
dep_rec, dep_eth, dep_mon, dep_repr = [latex_escape(clean_rdf(row_dep.get(k, ""))) for k in ["r", "e", "m", "p"]]

print("Final report queries completed. Paths cleaned and ready for LaTeX.")

EndPointInternalError: EndPointInternalError: The endpoint returned the HTTP status code 500. 

Response:
b'Entity pool initialization failure'

In [53]:
# ++++++++++++++++ FINAL UPDATED LATEX TEMPLATE +++++++++++++++++++++++++

latex_content = rf"""\documentclass[sigconf]{{acmart}}

\AtBeginDocument{{ \providecommand\BibTeX{{ Bib\TeX }} }}
\setcopyright{{acmlicensed}}
\copyrightyear{{2025}}
\acmYear{{2025}}
\acmDOI{{XXXXXXX.XXXXXXX}}

\acmConference[BI 2025]{{Business Intelligence Final Report}}{{-}}{{-}}

\begin{{document}}

\title{{BI2025 Final Report - Group {group_id}}}
%% ---Authors: Dynamically added ---
{author_block_latex}

\begin{{abstract}}
  this report documents the complete crisp-dm cycle for group {group_id} analyzing obesity levels.
  it covers business and data understanding, preparation, modeling using random forest,
  extensive evaluation including bias analysis, and deployment recommendations.
\end{{abstract}}

\ccsdesc[500]{{computing methodologies~machine learning}}
\keywords{{crisp-dm, provenance, random forest, obesity classification, bias analysis}}

\maketitle

%% --- 1. BUSINESS UNDERSTANDING ---
\section{{Business Understanding}}
\subsection{{Data Source and Scenario}} {bu_data_source}
\subsection{{Business Objectives}} {bu_objectives}
\subsection{{Business Success Criteria}} {bu_success_crit}
\subsection{{Data Mining Goals}} {bu_mining_goals}

%% --- 2. DATA UNDERSTANDING ---
\section{{Data Understanding}}
\subsection{{Dataset Overview}} {du_description}
\subsection{{Attribute Analysis}}
\begin{{table*}}[t]
  \caption{{dataset features}}
  \small
  \begin{{tabular}}{{p{{0.18\linewidth}}p{{0.12\linewidth}}p{{0.62\linewidth}}}}
    \toprule \textbf{{feature name}} & \textbf{{data type}} & \textbf{{description}} \\ \midrule
    {du_table_rows}
    \bottomrule
  \end{{tabular}}
\end{{table*}}
\subsection{{Statistical Properties}} {du_statistics_summary}
\subsection{{Data Quality}} {du_quality_summary}
\subsection{{visual exploration}}
{du_viz_summary}
\begin{{figure}}[h]
    \centering
    \includegraphics[width=0.8\linewidth]{{{du_viz_path}}}
    \caption{{visual analysis of obesity factors.}}
    \label{{fig:viz_2d}}
\end{{figure}}
\subsection{{Ethical Sensitivity}} {du_ethics_summary}

%% --- 3. DATA PREPARATION ---
\section{{Data Preparation}}
\subsection{{Applied Actions}} {dp_summary}
\subsection{{Rejected Steps}} {dp_rejected}
\subsection{{Derived Attributes}} {dp_derived}

%% --- 4. MODELING ---
\section{{Modeling}}
\subsection{{Algorithm Selection}}
{mod_algo_text}
\subsection{{Hyperparameter Identification and Tuning}}
{mod_hp_text}
\textit{{Note: tuning results visualized in the attached plots.}}
\subsection{{Data Splitting Strategy}}
{mod_split_text}
\subsection{{Final Model Retraining}}
{mod_retrain_text}

%% --- 5. EVALUATION ---
\section{{Evaluation}}
\subsection{{Final Test Performance}}
{eval_main_text}
\textbf{{Resulting Test Accuracy:}} {eval_perf_val}
\subsection{{Baseline and State-of-the-Art Comparison}}
{eval_baseline_text}
\subsection{{Detailed Performance Analysis}}
{eval_detailed_text}\
\subsection{{Success Criteria Assessment}}
{eval_success_text}\
\subsection{{Bias and Fairness Analysis}}
{eval_bias_text}\\


%% --- 6. DEPLOYMENT ---
\section{{Deployment}}
\subsection{{Recommendations}}
{dep_rec}
\subsection{{Ethical Risks}}
{dep_eth}
\subsection{{Monitoring and Maintenance}}
{dep_mon}
\subsection{{Reproducibility Reflection}}
{dep_repr}

\section{{Conclusion}}
the project successfully demonstrated the application of the crisp-dm process
to classify obesity levels with high accuracy. the provenance logging
ensures full transparency of all modeling and evaluation decisions.

\end{{document}}
"""

NameError: name 'author_block_latex' is not defined

The following includes the Latex report itself. It fills in the query-results from the cell before. The ACM Template is already filled. 
Make sure that you update Student A and B accordingly.

In [54]:
latex_content = rf"""\documentclass[sigconf]{{acmart}}

\AtBeginDocument{{ \providecommand\BibTeX{{ Bib\TeX }} }}
\setcopyright{{acmlicensed}}
\copyrightyear{{2025}}
\acmYear{{2025}}
\acmDOI{{XXXXXXX.XXXXXXX}}

\acmConference[BI 2025]{{Business Intelligence}}{{-}}{{-}}

\begin{{document}}

\title{{BI2025 Experiment Report - Group {group_id}}}
%% ---Authors: Dynamically added ---
{author_block_latex}

\begin{{abstract}}
  This report documents the machine learning experiment for Group {group_id}, following the CRISP-DM process model.
\end{{abstract}}

\ccsdesc[500]{{Computing methodologies~Machine learning}}
\keywords{{CRISP-DM, Provenance, Knowledge Graph, Machine Learning}}

\maketitle

%% --- 1. Business Understanding ---
\section{{Business Understanding}}

\subsection{{Data Source and Scenario}}
{bu_data_source}

\subsection{{Business Objectives}}
{bu_objectives}

%% --- 2. Data Understanding ---
\section{{Data Understanding}}
\textbf{{Dataset Description:}} {du_description}

The following features were identified in the dataset:

\begin{{table}}[h]
  \caption{{Raw Data Features}}
  \label{{tab:features}}
  \begin{{tabular}}{{lp{{0.2\linewidth}}p{{0.4\linewidth}}}}
    \toprule
    \textbf{{Feature Name}} & \textbf{{Data Type}} & \textbf{{Description}} \\
    \midrule
    {du_table_rows}
    \bottomrule
  \end{{tabular}}
\end{{table}}

%% --- 3. Data Preparation ---
\section{{Data Preparation}}
\subsection{{Data Cleaning}}
Describe your Data preparation steps here and include respective graph data.


%% --- 4. Modeling ---
\section{{Modeling}}

\subsection{{Hyperparameter Configuration}}
The model was trained using the following hyperparameter settings:

\begin{{table}}[h]
  \caption{{Hyperparameter Settings}}
  \label{{tab:hyperparams}}
  \begin{{tabular}}{{lp{{0.4\linewidth}}l}}
    \toprule
    \textbf{{Parameter}} & \textbf{{Description}} & \textbf{{Value}} \\
    \midrule
    {hp_table_rows}
    \bottomrule
  \end{{tabular}}
\end{{table}}

\subsection{{Training Run}}
A training run was executed with the following characteristics:
\begin{{itemize}}
    \item \textbf{{Algorithm:}} {mod_algo}
    \item \textbf{{Start Time:}} {mod_start}
    \item \textbf{{End Time:}} {mod_end}
    \item \textbf{{Result:}} {mod_m_lbl} = {mod_m_val}
\end{{itemize}}

%% --- 5. Evaluation ---
\section{{Evaluation}}

%% --- 6. Deployment ---
\section{{Deployment}}

\section{{Conclusion}}

\end{{document}}
"""

NameError: name 'author_block_latex' is not defined

In [ ]:
# This cell stores the Latex report to the data/report directory

out_dir = os.path.join("data", "report")
os.makedirs(out_dir, exist_ok=True)
out_path = os.path.join(out_dir, "experiment_report.tex")

with open(out_path, "w", encoding="utf-8") as f:
    f.write(latex_content)

print(f"Report written to: {out_path}")